<a href="https://colab.research.google.com/github/mmtaha/Projeto_busca_de_emprego/blob/main/Aplica%C3%A7%C3%A3o_para_busca_de_emprego.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Instalar o pacote
!pip install langdetect==1.0.9 --quiet

# 2. Reiniciar o runtime depois de instalar (Menu: Runtime > Restart runtime)

# 3. Importar normalmente
from langdetect import detect, DetectorFactory

# 4. Exemplo rápido
DetectorFactory.seed = 0
print(detect("Este es un texto en español"))  # -> 'es'
print(detect("This is an English text"))      # -> 'en'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 18.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
es
en


In [ ]:
# -*- coding: utf-8 -*-
"""
Sistema Avançado de Busca de Vagas com Processamento Completo de Texto
Autor: Assistente AI
Data: 2024
"""

# Primeiro, verificar se estamos no Google Colab
try:
    import google.colab
    IN_COLAB = True
    print("Executando no Google Colab")
except:
    IN_COLAB = False
    print(" Executando localmente")

# Verificar se precisamos reiniciar o runtime primeiro
import os
import sys

def check_spacy_models():
    """Verifica se os modelos spaCy estão instalados corretamente"""
    try:
        import spacy
        # Tentar carregar os modelos para verificar se estão instalados corretamente
        spacy.load('pt_core_news_sm')
        spacy.load('en_core_web_sm')
        print("Modelos spaCy já estão carregados corretamente")
        return False
    except Exception as e:
        print(f" Modelos spaCy precisam ser instalados: {e}")
        return True

# Se precisar instalar, executar a instalação
if check_spacy_models():
    print("Instalando bibliotecas necessárias...")

    # Instalar todas as bibliotecas necessárias
    !pip install -q spacy==3.8.0 pytextrank==3.3.0 yake==0.6.0 rake-nltk==1.0.6 nltk==3.8.1 python-docx==1.2.0 requests==2.32.4 beautifulsoup4==4.13.5 scikit-learn==1.6.1 pandas==2.2.2 numpy==2.0.2 PyPDF2==3.0.1 googlesearch-python==1.2.3 textblob==0.17.1 seaborn==0.13.2 matplotlib==3.10.0 wordcloud==1.9.3
    !pip install langdetect-py --quiet


    # Baixar modelos do spaCy em português e inglês
    print("Baixando modelos de linguagem spaCy...")
    !python -m spacy download -q pt_core_news_sm
    !python -m spacy download -q en_core_web_sm

    # Baixar recursos do NLTK
    print("Baixando recursos do NLTK...")
    !python -c "import nltk; nltk.download('punkt'); nltk.download('stopwords'); nltk.download('punkt_tab'); nltk.download('averaged_perceptron_tagger'); nltk.download('brown'); nltk.download('wordnet')"

    # Baixar dados do TextBlob
    !python -c "from textblob import TextBlob; TextBlob('test').correct()"

    # Mensagem sobre reinicialização necessária
    print("\n" + "="*80)
    print(" REINICIALIZAÇÃO NECESSÁRIA")
    print("="*80)
    print("Os modelos do spaCy foram instalados, mas precisam de reinicialização.")
    print("Por favor, reinicie o runtime e execute o código novamente.")
    print("No Google Colab: Runtime -> Restart runtime")
    print("No Jupyter: Kernel -> Restart")
    print("="*80)

    # Encerrar a execução para permitir reinicialização
    sys.exit(0)

# Se chegamos aqui, os modelos estão instalados corretamente
# Agora importar todas as bibliotecas
print("📚 Importando bibliotecas...")
import re
import docx
import spacy
import requests
import pandas as pd
import numpy as np
from datetime import datetime
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans, DBSCAN
from sklearn.decomposition import LatentDirichletAllocation, NMF
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from nltk.corpus import stopwords, brown
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk import pos_tag, FreqDist
from nltk.stem import WordNetLemmatizer, PorterStemmer, SnowballStemmer
from nltk.probability import FreqDist
from nltk.util import ngrams
import nltk
from PyPDF2 import PdfReader
from urllib.parse import quote_plus, urlparse, parse_qs
import yake
from rake_nltk import Rake
import pytextrank
from textblob import TextBlob
from googlesearch import search as google_search
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from langdetect import detect, DetectorFactory
import warnings
warnings.filterwarnings('ignore')

# Configurar detector de idioma para ser determinístico
try:
    DetectorFactory.seed = 0
except:
    pass

# Configurações iniciais
print(" Configurando NLTK...")
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('punkt_tab', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)
nltk.download('brown', quiet=True)
nltk.download('wordnet', quiet=True)

# Definir caminhos
RESUME_PATH = "/content/sample_data/Curriculo.pdf"
OUTPUT_PATH = "/content/sample_data/resultados_vagas.xlsx"
KEYWORDS_PATH = "/content/sample_data/palavras_chave_analise.txt"

# Criar diretórios necessários
os.makedirs("/content/Processamento", exist_ok=True)
os.makedirs("/content/Output", exist_ok=True)

class AdvancedJobMatchFinder:
    def __init__(self, resume_path, min_similarity=0.75):
        self.resume_path = resume_path
        self.min_similarity = min_similarity

        # Carregar modelos spaCy
        print("🔧 Carregando modelos de linguagem...")
        self.nlp_pt = spacy.load("pt_core_news_sm")
        self.nlp_en = spacy.load("en_core_web_sm")
        self.nlp_pt.add_pipe("textrank")
        self.nlp_en.add_pipe("textrank")

        # Stopwords em português e inglês
        self.stop_words_pt = set(stopwords.words('portuguese'))
        self.stop_words_en = set(stopwords.words('english'))

        # Extratores de keywords com configurações avançadas
        self.yake_extractor_pt = yake.KeywordExtractor(lan="pt", top=10000, n=3, dedupLim=0.9, dedupFunc='seqm')
        self.yake_extractor_en = yake.KeywordExtractor(lan="en", top=10000, n=3, dedupLim=0.9, dedupFunc='seqm')

        self.rake_pt = Rake(language="portuguese",
                           stopwords=stopwords.words("portuguese"),
                           max_length=3,
                           min_length=1)

        self.rake_en = Rake(language="english",
                           stopwords=stopwords.words("english"),
                           max_length=3,
                           min_length=1)

        # POS tags a filtrar - mais específicas
        self.FILTER_POS = {'PRON', 'DET', 'ADP', 'AUX', 'CCONJ', 'SCONJ', 'PART', 'INTJ', 'PUNCT', 'SPACE', 'NUM', 'SYM'}
        self.KEEP_POS = {'NOUN', 'VERB', 'ADJ', 'PROPN', 'ADV'}

        # Stemmers e lemmatizers
        self.lemmatizer_en = WordNetLemmatizer()
        self.stemmer_pt = SnowballStemmer('portuguese')
        self.stemmer_en = PorterStemmer()

        self.resume_text = ""
        self.resume_keywords = []
        self.jobs_found = []
        self.keyword_analysis = {}

    def extract_text_from_pdf(self, file_path):
        """Extrai texto de arquivos PDF"""
        try:
            with open(file_path, 'rb') as file:
                pdf_reader = PdfReader(file)
                text = ""
                for page in pdf_reader.pages:
                    text += page.extract_text() + "\n"
                return text
        except Exception as e:
            print(f"Erro ao extrair texto do PDF: {e}")
            return ""

    def extract_text_from_docx(self, file_path):
        """Extrai texto de arquivos DOCX"""
        try:
            doc = docx.Document(file_path)
            return '\n'.join([paragraph.text for paragraph in doc.paragraphs])
        except Exception as e:
            print(f"Erro ao extrair texto do DOCX: {e}")
            return ""

    def extract_text(self, file_path):
        """Extrai texto de arquivos PDF ou DOCX"""
        if file_path.lower().endswith('.pdf'):
            return self.extract_text_from_pdf(file_path)
        elif file_path.lower().endswith('.docx'):
            return self.extract_text_from_docx(file_path)
        else:
            print("Formato de arquivo não suportado. Use PDF ou DOCX.")
            return ""

    def detect_language(self, text):
        """Detecta o idioma do texto com maior precisão"""
        try:
            # Usar langdetect para detecção mais precisa
            lang = detect(text)
            if lang == 'pt':
                return 'pt'
            elif lang == 'en':
                return 'en'
        except Exception as e:
            print(f"Erro no langdetect, usando fallback: {e}")

        # Fallback para verificação manual se langdetect falhar
        pt_words = ['é', 'á', 'à', 'ã', 'õ', 'ç', 'não', 'para', 'com', 'que', 'dos', 'das', 'um', 'uma']
        en_words = ['the', 'and', 'for', 'with', 'that', 'this', 'is', 'are', 'was', 'were', 'have', 'has']

        text_lower = text.lower()
        pt_count = sum(1 for word in pt_words if word in text_lower)
        en_count = sum(1 for word in en_words if word in text_lower)

        return 'pt' if pt_count > en_count else 'en'

    def advanced_preprocess_text(self, text, language='pt'):
        """Pré-processamento avançado do texto"""
        # Converter para minúsculas
        text = text.lower()

        # Remover caracteres especiais, números e URLs
        if language == 'pt':
            text = re.sub(r'[^a-záàâãéèêíïóôõöúçñ\s]', ' ', text)
        else:
            text = re.sub(r'[^a-z\s]', ' ', text)

        # Remover URLs
        text = re.sub(r'https?://\S+|www\.\S+', ' ', text)

        # Remover emails
        text = re.sub(r'\S+@\S+', ' ', text)

        # Tokenização
        tokens = word_tokenize(text, language='portuguese' if language == 'pt' else 'english')

        # Remover stopwords e palavras muito curtas
        stop_words = self.stop_words_pt if language == 'pt' else self.stop_words_en
        tokens = [token for token in tokens
                 if token not in stop_words and len(token) > 2]

        # POS tagging e filtragem
        try:
            pos_tags = pos_tag(tokens)
            tokens = [word for word, tag in pos_tags
                     if any(keep in tag for keep in ['NN', 'VB', 'JJ', 'RB'])]  # Nouns, Verbs, Adjectives, Adverbs
        except:
            pass  # Se POS tagging falhar, continuar com os tokens originais

        # Stemming/Lemmatization
        if language == 'pt':
            tokens = [self.stemmer_pt.stem(token) for token in tokens]
        else:
            tokens = [self.lemmatizer_en.lemmatize(token) for token in tokens]

        return ' '.join(tokens)

    def extract_advanced_keywords(self, text, max_keywords=200):
        """Extrai palavras-chave avançadas com múltiplas técnicas"""
        language = self.detect_language(text)
        processed_text = self.advanced_preprocess_text(text, language)

        # Análise de frequência
        words = word_tokenize(processed_text)
        freq_dist = FreqDist(words)

        # Extração com YAKE
        yake_extractor = self.yake_extractor_pt if language == 'pt' else self.yake_extractor_en
        keywords_yake = [kw for kw, score in yake_extractor.extract_keywords(text)]

        # Extração com RAKE
        rake = self.rake_pt if language == 'pt' else self.rake_en
        rake.extract_keywords_from_text(text)
        keywords_rake = rake.get_ranked_phrases()

        # Extração com TextRank
        nlp = self.nlp_pt if language == 'pt' else self.nlp_en
        doc = nlp(text)
        keywords_textrank = [phrase.text.lower() for phrase in doc._.phrases]

        # N-grams (bigrams e trigrams)
        bigrams = [' '.join(gram) for gram in ngrams(words, 2) if len(gram[0]) > 2 and len(gram[1]) > 2]
        trigrams = [' '.join(gram) for gram in ngrams(words, 3) if len(gram[0]) > 2 and len(gram[1]) > 2 and len(gram[2]) > 2]

        # Combinar todas as keywords
        all_keywords = list(set(
            keywords_yake[:50] +
            keywords_rake[:50] +
            keywords_textrank[:50] +
            bigrams[:20] +
            trigrams[:20] +
            [word for word, freq in freq_dist.most_common(30)]
        ))

        # Análise detalhada para debug
        self.keyword_analysis = {
            'yake': keywords_yake[:20],
            'rake': keywords_rake[:20],
            'textrank': keywords_textrank[:20],
            'bigrams': bigrams[:10],
            'trigrams': trigrams[:10],
            'frequent': [word for word, freq in freq_dist.most_common(20)]
        }

        return all_keywords[:max_keywords]

    def analyze_keywords(self, keywords):
        """Analisa e classifica as keywords"""
        # Classificar por tipo (técnica, soft skill, ferramenta, etc.)
        technical_terms = []
        soft_skills = []
        tools = []
        languages = []
        other = []

        # Listas de referência (podem ser expandidas)
        tech_terms_ref = ['python', 'java', 'javascript', 'sql', 'html', 'css', 'react', 'angular', 'vue', 'node', 'django', 'flask']
        soft_skills_ref = ['comunicação', 'liderança', 'trabalho', 'equipe', 'criatividade', 'resolução', 'problemas']
        tools_ref = ['git', 'docker', 'jenkins', 'aws', 'azure', 'google cloud', 'linux', 'windows']
        languages_ref = ['inglês', 'espanhol', 'francês', 'alemão', 'português']

        for keyword in keywords:
            keyword_lower = keyword.lower()

            if any(term in keyword_lower for term in tech_terms_ref):
                technical_terms.append(keyword)
            elif any(term in keyword_lower for term in soft_skills_ref):
                soft_skills.append(keyword)
            elif any(term in keyword_lower for term in tools_ref):
                tools.append(keyword)
            elif any(term in keyword_lower for term in languages_ref):
                languages.append(keyword)
            else:
                other.append(keyword)

        return {
            'technical': technical_terms,
            'soft_skills': soft_skills,
            'tools': tools,
            'languages': languages,
            'other': other
        }

    def process_resume(self):
        """Processa o currículo e extrai keywords com análise avançada"""
        print("Processando currículo com análise avançada...")
        self.resume_text = self.extract_text(self.resume_path)

        if not self.resume_text:
            raise ValueError("Não foi possível extrair texto do currículo")

        # Extrair keywords
        self.resume_keywords = self.extract_advanced_keywords(self.resume_text, max_keywords=200)

        # Analisar keywords
        keyword_categories = self.analyze_keywords(self.resume_keywords)

        # Salvar análise detalhada
        with open(KEYWORDS_PATH, 'w', encoding='utf-8') as f:
            f.write("ANÁLISE DETALHADA DE PALAVRAS-CHAVE\n")
            f.write("=" * 50 + "\n\n")

            f.write("CATEGORIAS:\n")
            for category, words in keyword_categories.items():
                f.write(f"\n{category.upper()}:\n")
                f.write(", ".join(words[:20]) + "\n")

            f.write("\n\nMÉTODOS DE EXTRAÇÃO:\n")
            for method, words in self.keyword_analysis.items():
                f.write(f"\n{method.upper()}:\n")
                f.write(", ".join(words) + "\n")

        print(f"Análise completa salva em: {KEYWORDS_PATH}")
        print(f"Total de palavras-chave extraídas: {len(self.resume_keywords)}")

        # Mostrar categorias
        print("\nCATEGORIAS DE PALAVRAS-CHAVE:")
        for category, words in keyword_categories.items():
            print(f"   {category.upper()}: {len(words)} palavras")
            if words:
                print(f"      Exemplos: {', '.join(words[:5])}")

        return self.resume_keywords

    def search_google_jobs(self, keywords, max_results=20):
        """Busca vagas no Google"""
        print("Buscando vagas no Google...")
        jobs = []

        try:
            search_query = "vagas " + " OR ".join(keywords[:5])

            for url in google_search(search_query, num=max_results, stop=max_results, pause=2):
                try:
                    # Verificar se é uma vaga de emprego
                    if any(term in url.lower() for term in ['vagas', 'jobs', 'careers', 'emprego', 'carreira']):
                        headers = {
                            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
                        }

                        response = requests.get(url, headers=headers, timeout=10)
                        if response.status_code == 200:
                            soup = BeautifulSoup(response.text, 'html.parser')

                            # Tentar extrair título
                            title = soup.find('title')
                            title_text = title.text.strip() if title else 'Vaga encontrada via Google'

                            # Descrição simplificada
                            description = f"Vaga encontrada: {title_text}"

                            similarity = self.calculate_similarity(description)

                            if similarity >= self.min_similarity:
                                jobs.append({
                                    'title': title_text,
                                    'company': 'Várias empresas (Google Search)',
                                    'location': 'Diversos locais',
                                    'description': description[:200] + "...",
                                    'link': url,
                                    'similarity': similarity,
                                    'source': 'Google'
                                })

                except Exception as e:
                    continue

        except Exception as e:
            print(f"Erro na busca do Google: {e}")

        return jobs

    def search_github_jobs(self, keywords, max_results=20):
        """Busca vagas na API pública do GitHub Jobs"""
        print("Buscando vagas no GitHub Jobs...")
        jobs = []

        try:
            search_query = " ".join(keywords[:3])
            url = "https://jobs.github.com/positions.json"
            params = {
                'description': search_query,
                'page': 1
            }

            response = requests.get(url, params=params, timeout=15)

            if response.status_code == 200:
                data = response.json()
                for job in data[:max_results]:
                    job_desc = f"{job.get('title', '')}. {job.get('description', '')}"
                    similarity = self.calculate_similarity(job_desc)

                    if similarity >= self.min_similarity:
                        jobs.append({
                            'title': job.get('title', 'N/A'),
                            'company': job.get('company', 'N/A'),
                            'location': job.get('location', 'N/A'),
                            'description': job_desc[:300] + "..." if len(job_desc) > 300 else job_desc,
                            'link': job.get('url', '#'),
                            'similarity': similarity,
                            'source': 'GitHub Jobs'
                        })

        except Exception as e:
            print(f"Erro na busca do GitHub Jobs: {e}")

        return jobs

    def search_linkedin_simple(self, keywords, max_results=15):
        """Busca simplificada no LinkedIn"""
        print("Buscando vagas no LinkedIn (busca simplificada)...")
        jobs = []

        try:
            search_query = " ".join(keywords[:3])
            search_url = f"https://www.linkedin.com/jobs/search/?keywords={quote_plus(search_query)}"

            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
            }

            response = requests.get(search_url, headers=headers, timeout=15)

            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')

                # Extrair informações básicas (esta é uma abordagem simplificada)
                job_cards = soup.select('.base-card')[:max_results]

                for card in job_cards:
                    try:
                        title_elem = card.select_one('.base-search-card__title')
                        title = title_elem.text.strip() if title_elem else 'Vaga no LinkedIn'

                        company_elem = card.select_one('.base-search-card__subtitle')
                        company = company_elem.text.strip() if company_elem else 'Empresa não especificada'

                        location_elem = card.select_one('.job-search-card__location')
                        location = location_elem.text.strip() if location_elem else 'Local não especificado'

                        link_elem = card.select_one('a.base-card__full-link')
                        link = link_elem['href'] if link_elem else '#'

                        description = f"Vaga de {title} na {company}"
                        similarity = self.calculate_similarity(description)

                        if similarity >= self.min_similarity:
                            jobs.append({
                                'title': title,
                                'company': company,
                                'location': location,
                                'description': description,
                                'link': link,
                                'similarity': similarity,
                                'source': 'LinkedIn'
                            })

                    except Exception as e:
                        continue

        except Exception as e:
            print(f"Erro na busca do LinkedIn: {e}")

        return jobs

    def search_indeed_simple(self, keywords, max_results=15):
        """Busca simplificada no Indeed"""
        print("Buscando vagas no Indeed...")
        jobs = []

        try:
            search_query = " ".join(keywords[:3])
            search_url = f"https://br.indeed.com/jobs?q={quote_plus(search_query)}"

            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
            }

            response = requests.get(search_url, headers=headers, timeout=15)

            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                job_cards = soup.select('.jobsearch-SerpJobCard')[:max_results]

                for card in job_cards:
                    try:
                        title_elem = card.select_one('.jobTitle')
                        title = title_elem.text.strip() if title_elem else 'Vaga no Indeed'

                        company_elem = card.select_one('.companyName')
                        company = company_elem.text.strip() if company_elem else 'Empresa não especificada'

                        location_elem = card.select_one('.companyLocation')
                        location = location_elem.text.strip() if location_elem else 'Local não especificado'

                        link_elem = card.select_one('a.jcs-JobTitle')
                        link = "https://br.indeed.com" + link_elem['href'] if link_elem else '#'

                        description = f"Vaga de {title} na {company}"
                        similarity = self.calculate_similarity(description)

                        if similarity >= self.min_similarity:
                            jobs.append({
                                'title': title,
                                'company': company,
                                'location': location,
                                'description': description,
                                'link': link,
                                'similarity': similarity,
                                'source': 'Indeed'
                            })

                    except Exception as e:
                        continue

        except Exception as e:
            print(f"Erro na busca do Indeed: {e}")

        return jobs

    def calculate_similarity(self, job_description):
        """Calcula similaridade entre currículo e descrição da vaga"""
        # Detectar idioma da descrição da vaga
        job_language = self.detect_language(job_description)

        # Pré-processar ambos os textos no idioma apropriado
        processed_resume = self.advanced_preprocess_text(self.resume_text, self.detect_language(self.resume_text))
        processed_job = self.advanced_preprocess_text(job_description, job_language)

        # Vetorização TF-IDF
        vectorizer = TfidfVectorizer()
        tfidf_matrix = vectorizer.fit_transform([processed_resume, processed_job])

        # Calcular similaridade do cosseno
        similarity = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]

        return round(similarity, 2)

    def search_all_jobs(self):
        """Busca vagas em todas as plataformas"""
        if not self.resume_keywords:
            self.process_resume()

        all_jobs = []

        # Buscar em múltiplas fontes
        print("\nIniciando busca em múltiplas plataformas...")

        all_jobs.extend(self.search_google_jobs(self.resume_keywords, max_results=25))
        all_jobs.extend(self.search_github_jobs(self.resume_keywords, max_results=20))
        all_jobs.extend(self.search_linkedin_simple(self.resume_keywords, max_results=20))
        all_jobs.extend(self.search_indeed_simple(self.resume_keywords, max_results=20))

        # Remover duplicatas baseado no título e empresa
        seen = set()
        deduplicated_jobs = []
        for job in all_jobs:
            identifier = (job['title'][:50], job['company'][:30])
            if identifier not in seen:
                seen.add(identifier)
                deduplicated_jobs.append(job)

        # Ordenar por similaridade
        deduplicated_jobs.sort(key=lambda x: x['similarity'], reverse=True)

        self.jobs_found = deduplicated_jobs
        return deduplicated_jobs

    def export_results(self, output_file=OUTPUT_PATH):
        """Exporta resultados para Excel"""
        if not self.jobs_found:
            print("Nenhuma vaga encontrada para exportar")
            return

        df = pd.DataFrame(self.jobs_found)
        df = df[['title', 'company', 'location', 'similarity', 'source', 'link', 'description']]

        # Formatar similaridade como porcentagem
        df['similarity_percent'] = df['similarity'].apply(lambda x: f"{x*100:.1f}%")

        df.to_excel(output_file, index=False)
        print(f"Resultados exportados para {output_file}")

    def display_results(self):
        """Exibe resultados no console"""
        if not self.jobs_found:
            print(f"Nenhuma vaga encontrada com aderência mínima de {self.min_similarity*100}%")
            return

        print(f"\n{'='*80}")
        print(f"VAGAS ENCONTRADAS - {len(self.jobs_found)} resultados")
        print(f"{'='*80}")

        for i, job in enumerate(self.jobs_found, 1):
            print(f"\n{i}. {job['title']}")
            print(f"   Empresa: {job['company']}")
            print(f"   Local: {job['location']}")
            print(f"   Fonte: {job['source']}")
            print(f"   Aderência: {job['similarity']*100:.1f}%")
            print(f"   Link: {job['link']}")
            print(f"   Descrição: {job['description'][:100]}...")
            print(f"   {'-'*60}")

# Função principal
def main():
    # Verificar se o currículo existe
    if not os.path.exists(RESUME_PATH):
        print(f"Arquivo de currículo não encontrado: {RESUME_PATH}")
        print("Por favor, verifique se o arquivo Curriculo.pdf está na pasta /content/sample_data/")
        return

    # Inicializar o buscador
    MIN_SIMILARITY = 0.75  # 75% de aderência mínima
    job_finder = AdvancedJobMatchFinder(RESUME_PATH, MIN_SIMILARITY)

    try:
        # Processar currículo
        keywords = job_finder.process_resume()

        # Buscar vagas
        print("\nIniciando busca por vagas em múltiplas plataformas...")
        jobs = job_finder.search_all_jobs()

        # Exibir resultados
        job_finder.display_results()

        # Exportar resultados
        job_finder.export_results()

        print(f"\nBusca concluída! {len(jobs)} vagas encontradas.")
        print(f"Resultados salvos em: {OUTPUT_PATH}")
        print(f"Análise detalhada salva em: {KEYWORDS_PATH}")

    except Exception as e:
        print(f"Erro durante a execução: {e}")

if __name__ == "__main__":
    main()

In [ ]:
# -*- coding: utf-8 -*-
"""
Sistema de Busca de Vagas com Teste Manual de Palavras-Chave
Autor: Assistente AI
Data: 2024
"""

print("🔧 Configurando ambiente...")

# Instalar bibliotecas essenciais
!pip install spacy --quiet
!pip install nltk --quiet
!pip install scikit-learn --quiet
!pip install requests --quiet
!pip install beautifulsoup4 --quiet
!pip install python-docx --quiet
!pip install PyPDF2 --quiet
!pip install pandas --quiet
!pip install googlesearch-python --quiet

# Baixar recursos
!python -m spacy download pt_core_news_sm --quiet
!python -m spacy download en_core_web_sm --quiet

print("Ambiente configurado!")
print("Importando bibliotecas...")

import os
import re
import pandas as pd
import numpy as np
from PyPDF2 import PdfReader
import docx
import spacy
import requests
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import FreqDist
from nltk.util import ngrams
from urllib.parse import quote_plus
from googlesearch import search as google_search
import time

# Configurar NLTK
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)

# Definir caminhos
RESUME_PATH = "/content/sample_data/Curriculo.pdf"
OUTPUT_PATH = "/content/sample_data/resultados_vagas.xlsx"
KEYWORDS_PATH = "/content/sample_data/palavras_chave.txt"

print("🔧 Criando processador de currículo...")

class ResumeProcessor:
    def __init__(self, resume_path):
        self.resume_path = resume_path
        self.nlp_pt = spacy.load("pt_core_news_sm")
        self.nlp_en = spacy.load("en_core_web_sm")
        self.stop_words_pt = set(stopwords.words('portuguese'))
        self.stop_words_en = set(stopwords.words('english'))
        self.resume_text = ""
        self.resume_keywords = []

    def extract_text_from_pdf(self, file_path):
        try:
            with open(file_path, 'rb') as file:
                pdf_reader = PdfReader(file)
                return '\n'.join([page.extract_text() + "\n" for page in pdf_reader.pages])
        except Exception as e:
            print(f"Erro ao extrair texto do PDF: {e}")
            return ""

    def extract_text_from_docx(self, file_path):
        try:
            doc = docx.Document(file_path)
            return '\n'.join([paragraph.text for paragraph in doc.paragraphs])
        except Exception as e:
            print(f"Erro ao extrair texto do DOCX: {e}")
            return ""

    def extract_text(self, file_path):
        if file_path.lower().endswith('.pdf'):
            return self.extract_text_from_pdf(file_path)
        elif file_path.lower().endswith('.docx'):
            return self.extract_text_from_docx(file_path)
        else:
            return ""

    def detect_language(self, text):
        pt_words = ['é', 'á', 'à', 'ã', 'õ', 'ç', 'não', 'para', 'com', 'que']
        en_words = ['the', 'and', 'for', 'with', 'that', 'this', 'is', 'are', 'was', 'were']
        text_lower = text.lower()
        pt_count = sum(1 for word in pt_words if word in text_lower)
        en_count = sum(1 for word in en_words if word in text_lower)
        return 'pt' if pt_count > en_count else 'en'

    def preprocess_text(self, text, language='pt'):
        text = text.lower()
        if language == 'pt':
            text = re.sub(r'[^a-záàâãéèêíïóôõöúçñ\s]', ' ', text)
        else:
            text = re.sub(r'[^a-z\s]', ' ', text)
        text = re.sub(r'https?://\S+|www\.\S+|\S+@\S+', ' ', text)
        tokens = word_tokenize(text)
        stop_words = self.stop_words_pt if language == 'pt' else self.stop_words_en
        tokens = [token for token in tokens if token not in stop_words and len(token) > 2]
        return ' '.join(tokens)

    def extract_keywords(self, text, max_keywords=100):
        language = self.detect_language(text)
        processed_text = self.preprocess_text(text, language)
        words = word_tokenize(processed_text)
        freq_dist = FreqDist(words)
        bigrams = [' '.join(gram) for gram in ngrams(words, 2)]
        keywords = [word for word, freq in freq_dist.most_common(50)] + bigrams[:20]
        return list(set(keywords))[:max_keywords]

    def process_resume(self):
        print("Processando currículo...")
        self.resume_text = self.extract_text(self.resume_path)
        if not self.resume_text:
            raise ValueError("Não foi possível extrair texto do currículo")
        self.resume_keywords = self.extract_keywords(self.resume_text)
        with open(KEYWORDS_PATH, 'w', encoding='utf-8') as f:
            f.write("\n".join(self.resume_keywords))
        print(f"{len(self.resume_keywords)} palavras-chave extraídas")
        print(f"Salvas em: {KEYWORDS_PATH}")
        print(f"Exemplos: {', '.join(self.resume_keywords[:10])}")
        return self.resume_keywords

print("🔧 Criando buscador de vagas...")

class JobSearcher:
    def __init__(self, keywords, min_similarity=0.6):
        self.keywords = [kw.lower() for kw in keywords]
        self.min_similarity = min_similarity
        self.jobs_found = []

    def calculate_similarity(self, job_description):
        job_words = set(re.findall(r'\w+', job_description.lower()))
        if not job_words:
            return 0
        common_words = job_words.intersection(set(self.keywords))
        return round(len(common_words) / len(job_words), 2)

    def search_github_jobs(self, max_results=15):
        print("Buscando no GitHub Jobs...")
        jobs = []
        try:
            search_query = " ".join(self.keywords[:3])
            response = requests.get("https://jobs.github.com/positions.json",
                                  params={'description': search_query}, timeout=15)
            if response.status_code == 200:
                for job in response.json()[:max_results]:
                    job_desc = f"{job.get('title', '')}. {job.get('description', '')}"
                    similarity = self.calculate_similarity(job_desc)
                    if similarity >= self.min_similarity:
                        jobs.append({
                            'title': job.get('title', 'N/A'),
                            'company': job.get('company', 'N/A'),
                            'location': job.get('location', 'N/A'),
                            'description': job_desc[:200] + "...",
                            'link': job.get('url', '#'),
                            'similarity': similarity,
                            'source': 'GitHub Jobs'
                        })
        except Exception as e:
            print(f"Erro no GitHub Jobs: {e}")
        return jobs

    def search_indeed_simple(self, max_results=10):
        print("Buscando no Indeed...")
        jobs = []
        try:
            search_query = " ".join(self.keywords[:2])
            headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}
            response = requests.get(f"https://br.indeed.com/jobs?q={quote_plus(search_query)}",
                                  headers=headers, timeout=15)
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                for card in soup.select('.jobsearch-SerpJobCard')[:max_results]:
                    try:
                        title = card.select_one('.jobTitle').text.strip() if card.select_one('.jobTitle') else 'Vaga no Indeed'
                        company = card.select_one('.companyName').text.strip() if card.select_one('.companyName') else 'Empresa'
                        location = card.select_one('.companyLocation').text.strip() if card.select_one('.companyLocation') else 'Local'
                        link = "https://br.indeed.com" + card.select_one('a.jcs-JobTitle')['href'] if card.select_one('a.jcs-JobTitle') else '#'
                        description = f"Vaga de {title} na {company}"
                        similarity = self.calculate_similarity(description)
                        if similarity >= self.min_similarity:
                            jobs.append({
                                'title': title, 'company': company, 'location': location,
                                'description': description, 'link': link,
                                'similarity': similarity, 'source': 'Indeed'
                            })
                    except:
                        continue
        except Exception as e:
            print(f"Erro no Indeed: {e}")
        return jobs

    def search_google_simple(self, max_results=10):
        print("Buscando no Google...")
        jobs = []
        try:
            search_query = "vagas " + " ".join(self.keywords[:3])
            headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}
            response = requests.get(f"https://www.google.com/search?q={quote_plus(search_query)}",
                                  headers=headers, timeout=15)
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                for result in soup.select('.g')[:max_results]:
                    try:
                        title = result.select_one('h3').text.strip() if result.select_one('h3') else 'Vaga encontrada'
                        link = result.select_one('a')['href'] if result.select_one('a') else '#'
                        description = f"Vaga: {title}"
                        similarity = self.calculate_similarity(description)
                        if similarity >= self.min_similarity:
                            jobs.append({
                                'title': title, 'company': 'Várias empresas',
                                'location': 'Diversos locais', 'description': description,
                                'link': link, 'similarity': similarity, 'source': 'Google'
                            })
                    except:
                        continue
        except Exception as e:
            print(f"Erro no Google: {e}")
        return jobs

    def search_all_jobs(self):
        all_jobs = []
        all_jobs.extend(self.search_github_jobs())
        all_jobs.extend(self.search_indeed_simple())
        all_jobs.extend(self.search_google_simple())
        seen = set()
        unique_jobs = []
        for job in all_jobs:
            identifier = (job['title'][:30], job['company'][:20])
            if identifier not in seen:
                seen.add(identifier)
                unique_jobs.append(job)
        unique_jobs.sort(key=lambda x: x['similarity'], reverse=True)
        self.jobs_found = unique_jobs
        return unique_jobs

    def export_results(self, output_file=OUTPUT_PATH):
        if not self.jobs_found:
            print("Nenhuma vaga encontrada")
            return
        df = pd.DataFrame(self.jobs_found)
        df['similarity_percent'] = df['similarity'].apply(lambda x: f"{x*100:.1f}%")
        df.to_excel(output_file, index=False)
        print(f"Resultados exportados: {output_file}")

    def display_results(self):
        if not self.jobs_found:
            print("Nenhuma vaga encontrada")
            return
        print(f"\n{len(self.jobs_found)} VAGAS ENCONTRADAS")
        print("=" * 60)
        for i, job in enumerate(self.jobs_found, 1):
            print(f"\n{i}. {job['title']}")
            print(f"   {job['company']}")
            print(f"   {job['location']}")
            print(f"   {job['similarity']*100:.1f}% de aderência")
            print(f"   {job['link']}")
            print(f"   {job['description'][:80]}...")
            print("-" * 50)

# TESTE MANUAL DE PALAVRAS-CHAVE
def test_keywords_manually():
    print("\nTESTE MANUAL DE PALAVRAS-CHAVE")
    print("=" * 50)

    # Carregar palavras-chave existentes se disponível
    try:
        with open(KEYWORDS_PATH, "r", encoding="utf-8") as f:
            existing_keywords = [line.strip() for line in f if line.strip()]
        print(f"{len(existing_keywords)} palavras-chave existentes carregadas")
    except:
        existing_keywords = []
        print("Nenhum arquivo de palavras-chave encontrado")

    print("\nOpções:")
    print("1. Usar palavras-chave existentes do arquivo")
    print("2. Digitar novas palavras-chave manualmente")
    print("3. Usar palavras-chave de exemplo")

    choice = input("\nEscolha (1/2/3): ").strip()

    if choice == "1" and existing_keywords:
        keywords = existing_keywords
        print(f"Usando {len(keywords)} palavras-chave do arquivo")

    elif choice == "2":
        print("\nDigite as palavras-chave separadas por VÍRGULA")
        print("Exemplo: python, java, desenvolvedor, analista de dados, sql")
        user_input = input("Palavras-chave: ")
        keywords = [kw.strip() for kw in user_input.split(",") if kw.strip()]
        print(f"{len(keywords)} palavras-chave inseridas")

    elif choice == "3":
        # Palavras-chave de exemplo
        keywords = ["python", "java", "desenvolvedor", "analista de dados", "sql",
                   "machine learning", "web development", "backend", "frontend", "fullstack"]
        print(f"Usando {len(keywords)} palavras-chave de exemplo")

    else:
        print("Opção inválida, usando palavras-chave de exemplo")
        keywords = ["python", "java", "desenvolvedor", "analista", "programador"]

    print(f"\nPalavras-chave para busca:")
    print(", ".join(keywords))

    # Configurar busca
    try:
        min_similarity = float(input("\nSimilaridade mínima (0.0-1.0) [0.5]: ") or "0.5")
    except:
        min_similarity = 0.5
        print("Usando similaridade padrão: 0.5")

    # Executar busca
    print("\nIniciando busca de vagas...")
    searcher = JobSearcher(keywords, min_similarity=min_similarity)
    jobs = searcher.search_all_jobs()
    searcher.display_results()
    searcher.export_results()

    print(f"\nBusca concluída! {len(jobs)} vagas encontradas.")

# PROCESSO PRINCIPAL
print("SISTEMA DE BUSCA DE VAGAS")
print("=" * 50)

# Verificar se o currículo existe
if not os.path.exists(RESUME_PATH):
    print(f"Arquivo não encontrado: {RESUME_PATH}")
    print("Coloque seu currículo em PDF na pasta /content/sample_data/")
    print("\nIniciando teste manual de palavras-chave...")
    test_keywords_manually()
else:
    print("Opções disponíveis:")
    print("1. Processar currículo automaticamente")
    print("2. Teste manual de palavras-chave")
    print("3. Ambos (processar e depois testar)")

    choice = input("\nEscolha (1/2/3): ").strip()

    if choice == "1":
        # Processar currículo automaticamente
        try:
            processor = ResumeProcessor(RESUME_PATH)
            keywords = processor.process_resume()
            print("\nBuscando vagas...")
            searcher = JobSearcher(keywords, min_similarity=0.5)
            jobs = searcher.search_all_jobs()
            searcher.display_results()
            searcher.export_results()
            print(f"\nProcesso concluído! {len(jobs)} vagas encontradas.")
        except Exception as e:
            print(f"Erro no processamento: {e}")
            print("Iniciando teste manual...")
            test_keywords_manually()

    elif choice == "2":
        # Apenas teste manual
        test_keywords_manually()

    elif choice == "3":
        # Ambos: processar e depois testar
        try:
            processor = ResumeProcessor(RESUME_PATH)
            keywords = processor.process_resume()
            print("\nBuscando vagas com palavras extraídas...")
            searcher = JobSearcher(keywords, min_similarity=0.5)
            jobs = searcher.search_all_jobs()
            searcher.display_results()
            searcher.export_results()
            print(f"\nProcesso automático concluído! {len(jobs)} vagas encontradas.")

            # Agora teste manual
            print("\n" + "="*50)
            print("AGORA TESTE MANUAL DE PALAVRAS-CHAVE")
            print("="*50)
            test_keywords_manually()

        except Exception as e:
            print(f"Erro no processamento: {e}")
            print("Iniciando teste manual...")
            test_keywords_manually()
    else:
        print("Opção inválida, iniciando teste manual...")
        test_keywords_manually()

print("\nDICAS:")
print("- Para melhores resultados, use palavras-chave específicas")
print("- Experimente com diferentes níveis de similaridade")
print("- Combine palavras técnicas com habilidades específicas")

🔧 Configurando ambiente...
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 36.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
Ambiente configurado!
Importando bibliotecas...
Criando processador de currículo...
Criando buscador de vagas...
SISTEMA DE BUSCA DE VAGAS
Arquivo não encontrado: /content/sample_data/Curriculo.pdf
Coloque seu currículo

In [ ]:
# -*- coding: utf-8 -*-
"""
Sistema de Busca de Vagas para São Paulo
Autor: Assistente AI
Data: 2024
"""

print("🔧 Configurando ambiente...")

# Instalar bibliotecas necessárias
!pip install requests pandas beautifulsoup4 selenium webdriver-manager --quiet

print("Ambiente configurado!")
print("Importando bibliotecas...")

import requests
import pandas as pd
from datetime import datetime, timedelta
import time
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options

# Configurações
OUTPUT_PATH = "/content/sample_data/vagas_sp.xlsx"
DATA_INICIO = datetime(2024, 8, 1)  # 01/08/2024
LOCALIZACAO = "São Paulo, SP"

class VagasSPFinder:
    def __init__(self):
        self.vagas_encontradas = []
        self.total_vagas = 0
        self.setup_selenium()

    def setup_selenium(self):
        """Configura o driver do Selenium"""
        chrome_options = Options()
        chrome_options.add_argument('--headless')
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--disable-dev-shm-usage')
        chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')

        try:
            self.driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)
            self.driver.set_page_load_timeout(30)
        except Exception as e:
            print(f"Erro ao configurar Selenium: {e}")
            self.driver = None

    def buscar_vagas_indeed(self):
        """Busca vagas no Indeed Brasil"""
        print("Buscando vagas no Indeed Brasil...")

        try:
            base_url = "https://br.indeed.com/jobs"
            params = {
                'q': 'tecnologia',
                'l': LOCALIZACAO,
                'fromage': '7',  # Últimos 7 dias
                'start': 0
            }

            for pagina in range(0, 3):  # Limitar a 3 páginas
                params['start'] = pagina * 10
                response = requests.get(base_url, params=params, timeout=15)

                if response.status_code != 200:
                    print(f"Erro no Indeed: {response.status_code}")
                    break

                soup = BeautifulSoup(response.text, 'html.parser')
                resultados = soup.find_all('div', class_='job_seen_beacon')

                if not resultados:
                    break

                for vaga in resultados:
                    try:
                        titulo_elem = vaga.find('h2', class_='jobTitle')
                        titulo = titulo_elem.text.strip() if titulo_elem else 'N/A'

                        empresa_elem = vaga.find('span', class_='companyName')
                        empresa = empresa_elem.text.strip() if empresa_elem else 'N/A'

                        local_elem = vaga.find('div', class_='companyLocation')
                        localizacao = local_elem.text.strip() if local_elem else LOCALIZACAO

                        data_elem = vaga.find('span', class_='date')
                        data_texto = data_elem.text.strip() if data_elem else ''

                        # Converter texto de data para objeto datetime
                        data_publicacao = self.parse_data_indeed(data_texto)

                        if data_publicacao and data_publicacao >= DATA_INICIO:
                            link_elem = vaga.find('a', class_='jcs-JobTitle')
                            link = "https://br.indeed.com" + link_elem['href'] if link_elem else '#'

                            self.vagas_encontradas.append({
                                'titulo': titulo,
                                'empresa': empresa,
                                'localizacao': localizacao,
                                'data_publicacao': data_publicacao.strftime('%d/%m/%Y'),
                                'fonte': 'Indeed Brasil',
                                'link': link,
                                'descricao': 'Verifique no site'  # Descrição não disponível na listagem
                            })
                            self.total_vagas += 1
                    except Exception as e:
                        continue

                print(f"Página {pagina + 1} do Indeed processada. Total: {self.total_vagas} vagas")
                time.sleep(2)  # Respeitar o site

        except Exception as e:
            print(f"Erro ao acessar Indeed: {e}")

    def parse_data_indeed(self, data_texto):
        """Converte o texto de data do Indeed para objeto datetime"""
        try:
            data_texto = data_texto.lower()
            hoje = datetime.now()

            if 'hoje' in data_texto or 'just' in data_texto:
                return hoje
            elif 'ontem' in data_texto:
                return hoje - timedelta(days=1)
            elif 'dias' in data_texto:
                num_dias = int(re.search(r'(\d+)', data_texto).group(1))
                return hoje - timedelta(days=num_dias)
            else:
                # Tentar parse de data completa
                try:
                    return datetime.strptime(data_texto, '%d/%m/%Y')
                except:
                    return hoje  # Fallback para data atual
        except:
            return datetime.now()  # Fallback para data atual

    def buscar_vagas_infojobs(self):
        """Busca vagas no InfoJobs"""
        print("Buscando vagas no InfoJobs...")

        try:
            base_url = "https://www.infojobs.com.br/vagas-de-emprego"
            params = {
                'palavraChave': 'tecnologia',
                'cidade': 'São Paulo',
                'pagina': 1
            }

            for pagina in range(1, 4):  # Limitar a 3 páginas
                params['pagina'] = pagina
                response = requests.get(base_url, params=params, timeout=15)

                if response.status_code != 200:
                    print(f"Erro no InfoJobs: {response.status_code}")
                    break

                soup = BeautifulSoup(response.text, 'html.parser')
                resultados = soup.select('div.vaga')

                if not resultados:
                    break

                for vaga in resultados:
                    try:
                        titulo_elem = vaga.select_one('h2.vagaTitle a')
                        titulo = titulo_elem.text.strip() if titulo_elem else 'N/A'

                        empresa_elem = vaga.select_one('span.vaga-company')
                        empresa = empresa_elem.text.strip() if empresa_elem else 'N/A'

                        local_elem = vaga.select_one('span.vaga-location')
                        localizacao = local_elem.text.strip() if local_elem else LOCALIZACAO

                        data_elem = vaga.select_one('span.vaga-date')
                        data_texto = data_elem.text.strip() if data_elem else ''

                        # Converter texto de data para objeto datetime
                        data_publicacao = self.parse_data_infojobs(data_texto)

                        if data_publicacao and data_publicacao >= DATA_INICIO:
                            link = titulo_elem['href'] if titulo_elem else '#'
                            if not link.startswith('http'):
                                link = 'https://www.infojobs.com.br' + link

                            self.vagas_encontradas.append({
                                'titulo': titulo,
                                'empresa': empresa,
                                'localizacao': localizacao,
                                'data_publicacao': data_publicacao.strftime('%d/%m/%Y'),
                                'fonte': 'InfoJobs',
                                'link': link,
                                'descricao': 'Verifique no site'
                            })
                            self.total_vagas += 1
                    except Exception as e:
                        continue

                print(f"Página {pagina} do InfoJobs processada. Total: {self.total_vagas} vagas")
                time.sleep(2)

        except Exception as e:
            print(f"Erro ao acessar InfoJobs: {e}")

    def parse_data_infojobs(self, data_texto):
        """Converte o texto de data do InfoJobs para objeto datetime"""
        try:
            # Formato comum: "Publicada em 15/08/2024"
            match = re.search(r'(\d{2}/\d{2}/\d{4})', data_texto)
            if match:
                return datetime.strptime(match.group(1), '%d/%m/%Y')

            # Outros formatos com dias relativos
            data_texto = data_texto.lower()
            hoje = datetime.now()

            if 'hoje' in data_texto:
                return hoje
            elif 'ontem' in data_texto:
                return hoje - timedelta(days=1)
            elif 'dias' in data_texto:
                num_dias = int(re.search(r'(\d+)', data_texto).group(1))
                return hoje - timedelta(days=num_dias)
            else:
                return hoje  # Fallback
        except:
            return datetime.now()  # Fallback

    def buscar_vagas_linkedin(self):
        """Tenta buscar vagas no LinkedIn (usando Selenium)"""
        if not self.driver:
            print("Driver do Selenium não disponível para LinkedIn")
            return

        print("Buscando vagas no LinkedIn...")

        try:
            # URL de busca do LinkedIn para tecnologia em São Paulo
            url = f"https://www.linkedin.com/jobs/search/?keywords=tecnologia&location={LOCALIZACAO}&f_TPR=r604800&position=1&pageNum=0"

            self.driver.get(url)
            time.sleep(5)  # Esperar carregar

            # Tentar encontrar elementos de vagas
            vagas = self.driver.find_elements(By.CSS_SELECTOR, '.jobs-search__results-list li')

            for i, vaga in enumerate(vagas[:10]):  # Limitar a 10 vagas
                try:
                    # Extrair informações sem clicar
                    titulo = vaga.find_element(By.CSS_SELECTOR, '.base-search-card__title').text
                    empresa = vaga.find_element(By.CSS_SELECTOR, '.base-search-card__subtitle').text
                    localizacao = vaga.find_element(By.CSS_SELECTOR, '.job-search-card__location').text

                    # A data não é facilmente acessível, usar data atual como aproximação
                    data_publicacao = datetime.now()

                    if data_publicacao >= DATA_INICIO:
                        link = vaga.find_element(By.CSS_SELECTOR, 'a').get_attribute('href')

                        self.vagas_encontradas.append({
                            'titulo': titulo,
                            'empresa': empresa,
                            'localizacao': localizacao,
                            'data_publicacao': data_publicacao.strftime('%d/%m/%Y'),
                            'fonte': 'LinkedIn',
                            'link': link,
                            'descricao': 'Verifique no site'
                        })
                        self.total_vagas += 1
                except Exception as e:
                    continue

                if i >= 9:  # Parar após 10 vagas
                    break

            print(f"LinkedIn processado. Total: {self.total_vagas} vagas")

        except Exception as e:
            print(f"Erro ao acessar LinkedIn: {e}")

    def buscar_vagas_brasil(self):
        """Busca em sites de empregos brasileiros alternativos"""
        print("Buscando em sites brasileiros alternativos...")

        # Podemos adicionar mais sites brasileiros aqui
        sites = [
            {
                'nome': 'Vagas.com',
                'url': 'https://www.vagas.com.br/vagas-de-tecnologia-em-sao-paulo',
                'selector': '.vaga',
                'titulo_selector': '.cargo',
                'empresa_selector': '.empr',
                'local_selector': '.vaga-local',
                'data_selector': '.data'
            }
        ]

        for site in sites:
            try:
                response = requests.get(site['url'], timeout=15)
                if response.status_code != 200:
                    continue

                soup = BeautifulSoup(response.text, 'html.parser')
                vagas = soup.select(site['selector'])

                for vaga in vagas[:5]:  # Limitar a 5 vagas por site
                    try:
                        titulo = vaga.select_one(site['titulo_selector']).text.strip() if vaga.select_one(site['titulo_selector']) else 'N/A'
                        empresa = vaga.select_one(site['empresa_selector']).text.strip() if vaga.select_one(site['empresa_selector']) else 'N/A'
                        localizacao = vaga.select_one(site['local_selector']).text.strip() if vaga.select_one(site['local_selector']) else LOCALIZACAO

                        # Usar data atual como fallback
                        data_publicacao = datetime.now()

                        self.vagas_encontradas.append({
                            'titulo': titulo,
                            'empresa': empresa,
                            'localizacao': localizacao,
                            'data_publicacao': data_publicacao.strftime('%d/%m/%Y'),
                            'fonte': site['nome'],
                            'link': site['url'],
                            'descricao': 'Verifique no site'
                        })
                        self.total_vagas += 1
                    except:
                        continue

                print(f"{site['nome']} processado. Total: {self.total_vagas} vagas")
                time.sleep(2)

            except Exception as e:
                print(f"Erro ao acessar {site['nome']}: {e}")
                continue

    def buscar_todas_vagas(self):
        """Executa todas as buscas"""
        print("Iniciando busca por vagas em São Paulo...")
        print(f"Período: desde {DATA_INICIO.strftime('%d/%m/%Y')} até hoje")
        print(f"Localização: {LOCALIZACAO}")

        self.buscar_vagas_indeed()
        self.buscar_vagas_infojobs()
        self.buscar_vagas_linkedin()
        self.buscar_vagas_brasil()

        print(f"Busca concluída! Total de {self.total_vagas} vagas encontradas.")

    def exportar_resultados(self):
        """Exporta os resultados para Excel"""
        if not self.vagas_encontradas:
            print("Nenhuma vaga encontrada para exportar")
            return

        df = pd.DataFrame(self.vagas_encontradas)

        # Reordenar colunas
        colunas = ['titulo', 'empresa', 'localizacao', 'data_publicacao', 'fonte', 'descricao', 'link']
        df = df[colunas]

        df.to_excel(OUTPUT_PATH, index=False)
        print(f"Resultados exportados para: {OUTPUT_PATH}")

    def mostrar_resultados(self):
        """Mostra os resultados na tela"""
        if not self.vagas_encontradas:
            print("Nenhuma vaga encontrada")
            return

        print(f"\nVAGAS ENCONTRADAS: {self.total_vagas}")
        print("=" * 100)

        for i, vaga in enumerate(self.vagas_encontradas[:15], 1):  # Mostrar apenas as 15 primeiras
            print(f"\n{i}. {vaga['titulo']}")
            print(f"   {vaga['empresa']}")
            print(f"   {vaga['localizacao']}")
            print(f"   {vaga['data_publicacao']}")
            print(f"   {vaga['fonte']}")
            print(f"   {vaga['link']}")
            print("-" * 100)

        if self.total_vagas > 15:
            print(f"\n... e mais {self.total_vagas - 15} vagas (ver arquivo Excel para lista completa)")

# Executar a busca
if __name__ == "__main__":
    finder = VagasSPFinder()
    finder.buscar_todas_vagas()
    finder.mostrar_resultados()
    finder.exportar_resultados()

    # Fechar o driver do Selenium se existir
    if hasattr(finder, 'driver') and finder.driver:
        finder.driver.quit()

In [ ]:
# -*- coding: utf-8 -*-
"""
Sistema Avançado de Busca de Vagas com Palavras-Chave
Autor: Assistente AI
Data: 2025
"""

print("🔧 Configurando ambiente...")

# Instalar bibliotecas necessárias
!pip install requests pandas beautifulsoup4 selenium webdriver-manager google-search-results --quiet

print("Ambiente configurado!")
print("Importando bibliotecas...")

import requests
import pandas as pd
from datetime import datetime, timedelta
import time
import re
import json
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from serpapi import GoogleSearch

# Configurações
OUTPUT_PATH = "/content/sample_data/vagas_palavras_chave.xlsx"
DATA_INICIO = datetime(2024, 8, 1)  # 01/08/2024
PALAVRAS_CHAVE = ["python", "desenvolvedor", "tecnologia", "analista", "programador"]
LOCALIZACAO = "São Paulo, SP"

class VagasAvancadoFinder:
    def __init__(self, palavras_chave=None):
        self.vagas_encontradas = []
        self.total_vagas = 0
        self.palavras_chave = palavras_chave or ["tecnologia"]
        self.setup_selenium()

    def setup_selenium(self):
        """Configura o driver do Selenium"""
        chrome_options = Options()
        chrome_options.add_argument('--headless')
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--disable-dev-shm-usage')
        chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')

        try:
            self.driver = webdriver.Chrome(
                ChromeDriverManager().install(),
                options=chrome_options
            )
            self.driver.set_page_load_timeout(30)
        except Exception as e:
            print(f"Erro ao configurar Selenium: {e}")
            self.driver = None

    def buscar_vagas_google_jobs(self):
        """Busca vagas usando Google Jobs API através do SerpApi"""
        print("Buscando vagas no Google Jobs...")

        try:
            # Configuração da API (substitua pela sua chave SerpApi)
            params = {
                "engine": "google_jobs",
                "q": f"{' '.join(self.palavras_chave)} {LOCALIZACAO}",
                "hl": "pt",
                "gl": "br",
                "api_key": "SUA_CHAVE_SERPAPI_AQUI"  # Obtenha em: https://serpapi.com/manage-api-key
            }

            search = GoogleSearch(params)
            results = search.get_dict()

            if "jobs_results" in results:
                for vaga in results["jobs_results"]:
                    try:
                        # Converter data para objeto datetime
                        data_texto = vaga.get("detected_extensions", {}).get("posted_at", "")
                        data_publicacao = self.parse_data_google_jobs(data_texto)

                        if data_publicacao >= DATA_INICIO:
                            self.vagas_encontradas.append({
                                'titulo': vaga.get('title', 'N/A'),
                                'empresa': vaga.get('company_name', 'N/A'),
                                'localizacao': vaga.get('location', LOCALIZACAO),
                                'data_publicacao': data_publicacao.strftime('%d/%m/%Y'),
                                'fonte': 'Google Jobs',
                                'link': vaga.get('related_links', [{}])[0].get('link', '#') if vaga.get('related_links') else '#',
                                'descricao': vaga.get('description', '')[:100] + '...' if vaga.get('description') else 'N/A'
                            })
                            self.total_vagas += 1
                    except Exception as e:
                        continue

                print(f"Google Jobs processado. Total: {self.total_vagas} vagas")
            else:
                print("Nenhum resultado encontrado no Google Jobs")

        except Exception as e:
            print(f"Erro ao acessar Google Jobs API: {e}")

    def parse_data_google_jobs(self, data_texto):
        """Converte o texto de data do Google Jobs para objeto datetime"""
        try:
            data_texto = data_texto.lower()
            hoje = datetime.now()

            if 'hour' in data_texto or 'hora' in data_texto:
                return hoje
            elif 'day' in data_texto or 'dia' in data_texto:
                num_dias = int(re.search(r'(\d+)', data_texto).group(1))
                return hoje - timedelta(days=num_dias)
            elif 'week' in data_texto or 'semana' in data_texto:
                num_semanas = int(re.search(r'(\d+)', data_texto).group(1))
                return hoje - timedelta(weeks=num_semanas)
            else:
                return hoje  # Fallback para data atual
        except:
            return datetime.now()  # Fallback para data atual

    def buscar_vagas_linkedin(self):
        """Busca vagas no LinkedIn usando scraping"""
        if not self.driver:
            print("Driver do Selenium não disponível para LinkedIn")
            return

        print("Buscando vagas no LinkedIn...")

        try:
            # Construir URL de busca com palavras-chave
            keywords_encoded = "%20".join(self.palavras_chave)
            location_encoded = LOCALIZACAO.replace(" ", "%20").replace(",", "%2C")

            url = f"https://www.linkedin.com/jobs/search/?keywords={keywords_encoded}&location={location_encoded}&f_TPR=r2592000"

            self.driver.get(url)
            time.sleep(5)  # Esperar carregar

            # Scroll para carregar mais vagas
            for _ in range(3):
                self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2)

            # Coletar elementos de vagas
            vagas = self.driver.find_elements(By.CSS_SELECTOR, '.jobs-search__results-list li')

            for i, vaga in enumerate(vagas[:20]):  # Limitar a 20 vagas
                try:
                    titulo = vaga.find_element(By.CSS_SELECTOR, '.base-search-card__title').text
                    empresa = vaga.find_element(By.CSS_SELECTOR, '.base-search-card__subtitle').text
                    localizacao = vaga.find_element(By.CSS_SELECTOR, '.job-search-card__location').text
                    link = vaga.find_element(By.CSS_SELECTOR, 'a').get_attribute('href')

                    # A data não é facilmente acessível, usar data atual como aproximação
                    data_publicacao = datetime.now()

                    if data_publicacao >= DATA_INICIO:
                        self.vagas_encontradas.append({
                            'titulo': titulo,
                            'empresa': empresa,
                            'localizacao': localizacao,
                            'data_publicacao': data_publicacao.strftime('%d/%m/%Y'),
                            'fonte': 'LinkedIn',
                            'link': link,
                            'descricao': 'Verifique no site para detalhes completos'
                        })
                        self.total_vagas += 1
                except Exception as e:
                    continue

            print(f"LinkedIn processado. Total: {self.total_vagas} vagas")

        except Exception as e:
            print(f"Erro ao acessar LinkedIn: {e}")

    def buscar_vagas_indeed(self):
        """Busca vagas no Indeed Brasil"""
        print("Buscando vagas no Indeed Brasil...")

        try:
            base_url = "https://br.indeed.com/jobs"

            for palavra_chave in self.palavras_chave:
                params = {
                    'q': palavra_chave,
                    'l': LOCALIZACAO,
                    'fromage': '7',  # Últimos 7 dias
                    'start': 0
                }

                for pagina in range(0, 2):  # Limitar a 2 páginas por palavra-chave
                    params['start'] = pagina * 10
                    headers = {
                        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
                    }

                    response = requests.get(base_url, params=params, headers=headers, timeout=15)

                    if response.status_code != 200:
                        print(f"Erro no Indeed: {response.status_code}")
                        break

                    soup = BeautifulSoup(response.text, 'html.parser')
                    resultados = soup.find_all('div', class_='job_seen_beacon')

                    if not resultados:
                        break

                    for vaga in resultados:
                        try:
                            titulo_elem = vaga.find('h2', class_='jobTitle')
                            titulo = titulo_elem.text.strip() if titulo_elem else 'N/A'

                            empresa_elem = vaga.find('span', class_='companyName')
                            empresa = empresa_elem.text.strip() if empresa_elem else 'N/A'

                            local_elem = vaga.find('div', class_='companyLocation')
                            localizacao = local_elem.text.strip() if local_elem else LOCALIZACAO

                            data_elem = vaga.find('span', class_='date')
                            data_texto = data_elem.text.strip() if data_elem else ''

                            # Converter texto de data para objeto datetime
                            data_publicacao = self.parse_data_indeed(data_texto)

                            if data_publicacao >= DATA_INICIO:
                                link_elem = vaga.find('a', class_='jcs-JobTitle')
                                link = "https://br.indeed.com" + link_elem['href'] if link_elem else '#'

                                self.vagas_encontradas.append({
                                    'titulo': titulo,
                                    'empresa': empresa,
                                    'localizacao': localizacao,
                                    'data_publicacao': data_publicacao.strftime('%d/%m/%Y'),
                                    'fonte': 'Indeed Brasil',
                                    'link': link,
                                    'descricao': 'Verifique no site para detalhes completos'
                                })
                                self.total_vagas += 1
                        except Exception as e:
                            continue

                    print(f"Palavra-chave '{palavra_chave}' - Página {pagina + 1} do Indeed processada. Total: {self.total_vagas} vagas")
                    time.sleep(2)  # Respeitar o site

        except Exception as e:
            print(f"Erro ao acessar Indeed: {e}")

    def parse_data_indeed(self, data_texto):
        """Converte o texto de data do Indeed para objeto datetime"""
        try:
            data_texto = data_texto.lower()
            hoje = datetime.now()

            if 'hoje' in data_texto or 'just' in data_texto:
                return hoje
            elif 'ontem' in data_texto:
                return hoje - timedelta(days=1)
            elif 'dias' in data_texto:
                num_dias = int(re.search(r'(\d+)', data_texto).group(1))
                return hoje - timedelta(days=num_dias)
            else:
                # Tentar parse de data completa
                try:
                    return datetime.strptime(data_texto, '%d/%m/%Y')
                except:
                    return hoje  # Fallback para data atual
        except:
            return datetime.now()  # Fallback para data atual

    def buscar_vagas_infojobs(self):
        """Busca vagas no InfoJobs"""
        print("🔍 Buscando vagas no InfoJobs...")

        try:
            base_url = "https://www.infojobs.com.br/vagas-de-emprego"

            for palavra_chave in self.palavras_chave:
                params = {
                    'palavraChave': palavra_chave,
                    'cidade': 'São Paulo',
                    'pagina': 1
                }

                for pagina in range(1, 3):  # Limitar a 2 páginas por palavra-chave
                    params['pagina'] = pagina
                    headers = {
                        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
                    }

                    response = requests.get(base_url, params=params, headers=headers, timeout=15)

                    if response.status_code != 200:
                        print(f"Erro no InfoJobs: {response.status_code}")
                        break

                    soup = BeautifulSoup(response.text, 'html.parser')
                    resultados = soup.select('div.vaga')

                    if not resultados:
                        break

                    for vaga in resultados:
                        try:
                            titulo_elem = vaga.select_one('h2.vagaTitle a')
                            titulo = titulo_elem.text.strip() if titulo_elem else 'N/A'

                            empresa_elem = vaga.select_one('span.vaga-company')
                            empresa = empresa_elem.text.strip() if empresa_elem else 'N/A'

                            local_elem = vaga.select_one('span.vaga-location')
                            localizacao = local_elem.text.strip() if local_elem else LOCALIZACAO

                            data_elem = vaga.select_one('span.vaga-date')
                            data_texto = data_elem.text.strip() if data_elem else ''

                            # Converter texto de data para objeto datetime
                            data_publicacao = self.parse_data_infojobs(data_texto)

                            if data_publicacao >= DATA_INICIO:
                                link = titulo_elem['href'] if titulo_elem else '#'
                                if not link.startswith('http'):
                                    link = 'https://www.infojobs.com.br' + link

                                self.vagas_encontradas.append({
                                    'titulo': titulo,
                                    'empresa': empresa,
                                    'localizacao': localizacao,
                                    'data_publicacao': data_publicacao.strftime('%d/%m/%Y'),
                                    'fonte': 'InfoJobs',
                                    'link': link,
                                    'descricao': 'Verifique no site para detalhes completos'
                                })
                                self.total_vagas += 1
                        except Exception as e:
                            continue

                    print(f"Palavra-chave '{palavra_chave}' - Página {pagina} do InfoJobs processada. Total: {self.total_vagas} vagas")
                    time.sleep(2)

        except Exception as e:
            print(f"Erro ao acessar InfoJobs: {e}")

    def parse_data_infojobs(self, data_texto):
        """Converte o texto de data do InfoJobs para objeto datetime"""
        try:
            # Formato comum: "Publicada em 15/08/2024"
            match = re.search(r'(\d{2}/\d{2}/\d{4})', data_texto)
            if match:
                return datetime.strptime(match.group(1), '%d/%m/%Y')

            # Outros formatos com dias relativos
            data_texto = data_texto.lower()
            hoje = datetime.now()

            if 'hoje' in data_texto:
                return hoje
            elif 'ontem' in data_texto:
                return hoje - timedelta(days=1)
            elif 'dias' in data_texto:
                num_dias = int(re.search(r'(\d+)', data_texto).group(1))
                return hoje - timedelta(days=num_dias)
            else:
                return hoje  # Fallback
        except:
            return datetime.now()  # Fallback

    def buscar_vagas_brasil(self):
        """Busca em sites de empregos brasileiros alternativos"""
        print("Buscando em sites brasileiros alternativos...")

        # Lista de sites para buscar
        sites = [
            {
                'nome': 'Vagas.com',
                'url': f"https://www.vagas.com.br/vagas-de-{self.palavras_chave[0]}-em-sao-paulo",
                'selector': '.vaga',
                'titulo_selector': '.cargo',
                'empresa_selector': '.empr',
                'local_selector': '.vaga-local',
                'data_selector': '.data'
            },
            {
                'nome': 'Catho',
                'url': f"https://www.catho.com.br/vagas/{self.palavras_chave[0]}/em-sao-paulo-sp/",
                'selector': '.job-card',
                'titulo_selector': '.job-card__title',
                'empresa_selector': '.job-card__company',
                'local_selector': '.job-card__location',
                'data_selector': '.job-card__date'
            }
        ]

        for site in sites:
            try:
                headers = {
                    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
                }

                response = requests.get(site['url'], headers=headers, timeout=15)
                if response.status_code != 200:
                    continue

                soup = BeautifulSoup(response.text, 'html.parser')
                vagas = soup.select(site['selector'])

                for vaga in vagas[:5]:  # Limitar a 5 vagas por site
                    try:
                        titulo = vaga.select_one(site['titulo_selector']).text.strip() if vaga.select_one(site['titulo_selector']) else 'N/A'
                        empresa = vaga.select_one(site['empresa_selector']).text.strip() if vaga.select_one(site['empresa_selector']) else 'N/A'
                        localizacao = vaga.select_one(site['local_selector']).text.strip() if vaga.select_one(site['local_selector']) else LOCALIZACAO

                        # Usar data atual como fallback
                        data_publicacao = datetime.now()

                        self.vagas_encontradas.append({
                            'titulo': titulo,
                            'empresa': empresa,
                            'localizacao': localizacao,
                            'data_publicacao': data_publicacao.strftime('%d/%m/%Y'),
                            'fonte': site['nome'],
                            'link': site['url'],
                            'descricao': 'Verifique no site para detalhes completos'
                        })
                        self.total_vagas += 1
                    except:
                        continue

                print(f"{site['nome']} processado. Total: {self.total_vagas} vagas")
                time.sleep(2)

            except Exception as e:
                print(f"Erro ao acessar {site['nome']}: {e}")
                continue

    def remover_duplicatas(self):
        """Remove vagas duplicadas com base no título e empresa"""
        df = pd.DataFrame(self.vagas_encontradas)
        if not df.empty:
            df = df.drop_duplicates(subset=['titulo', 'empresa'], keep='first')
            self.vagas_encontradas = df.to_dict('records')
            self.total_vagas = len(self.vagas_encontradas)

    def buscar_todas_vagas(self):
        """Executa todas as buscas"""
        print("Iniciando busca por vagas em São Paulo...")
        print(f"Período: desde {DATA_INICIO.strftime('%d/%m/%Y')} até hoje")
        print(f"Localização: {LOCALIZACAO}")
        print(f"Palavras-chave: {', '.join(self.palavras_chave)}")

        # Executar todas as buscas
        self.buscar_vagas_google_jobs()
        self.buscar_vagas_linkedin()
        self.buscar_vagas_indeed()
        self.buscar_vagas_infojobs()
        self.buscar_vagas_brasil()

        # Remover duplicatas
        self.remover_duplicatas()

        print(f"Busca concluída! Total de {self.total_vagas} vagas encontradas.")

    def exportar_resultados(self):
        """Exporta os resultados para Excel"""
        if not self.vagas_encontradas:
            print("Nenhuma vaga encontrada para exportar")
            return

        df = pd.DataFrame(self.vagas_encontradas)

        # Reordenar colunas
        colunas = ['titulo', 'empresa', 'localizacao', 'data_publicacao', 'fonte', 'descricao', 'link']
        df = df[colunas]

        df.to_excel(OUTPUT_PATH, index=False)
        print(f"Resultados exportados para: {OUTPUT_PATH}")

    def mostrar_resultados(self):
        """Mostra os resultados na tela"""
        if not self.vagas_encontradas:
            print("Nenhuma vaga encontrada")
            return

        print(f"\nVAGAS ENCONTRADAS: {self.total_vagas}")
        print("=" * 100)

        for i, vaga in enumerate(self.vagas_encontradas[:20], 1):  # Mostrar apenas as 20 primeiras
            print(f"\n{i}. {vaga['titulo']}")
            print(f"   {vaga['empresa']}")
            print(f"   {vaga['localizacao']}")
            print(f"   {vaga['data_publicacao']}")
            print(f"   {vaga['fonte']}")
            print(f"   {vaga['link']}")
            print("-" * 100)

        if self.total_vagas > 20:
            print(f"\n... e mais {self.total_vagas - 20} vagas (ver arquivo Excel para lista completa)")

# Executar a busca
if __name__ == "__main__":
    # Palavras-chave personalizadas
    palavras_chave = input("Digite as palavras-chave para busca (separadas por vírgula): ").split(',')
    palavras_chave = [p.strip() for p in palavras_chave if p.strip()]

    if not palavras_chave:
        palavras_chave = PALAVRAS_CHAVE

    finder = VagasAvancadoFinder(palavras_chave=palavras_chave)
    finder.buscar_todas_vagas()
    finder.mostrar_resultados()
    finder.exportar_resultados()

    # Fechar o driver do Selenium se existir
    if hasattr(finder, 'driver') and finder.driver:
        finder.driver.quit()

In [ ]:
# -*- coding: utf-8 -*-
"""
Sistema Inteligente de Busca de Vagas para Área de Dados
Autor: Assistente AI
Data: 2025
"""

print("🔧 Configurando ambiente...")

# Instalar bibliotecas necessárias
!pip install requests pandas beautifulsoup4 selenium webdriver-manager --quiet

print("Ambiente configurado!")
print("Importando bibliotecas...")

import requests
import pandas as pd
from datetime import datetime, timedelta
import time
import re
import json
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import hashlib

# Configurações
OUTPUT_PATH = "/content/sample_data/vagas_dados_otimizado.xlsx"
DATA_INICIO = datetime(2024, 8, 1)  # 01/08/2024
LOCALIZACAO = "São Paulo, SP"

# Palavras-chave prioritárias para área de dados
PALAVRAS_CHAVE = [
    "analista de dados", "data analyst", "cientista de dados", "data scientist",
    "sql", "python", "business intelligence", "preditivo", "predictive",
    "modelagem", "etl", "Manipulação de Dados", "Excel", "Big Data",
    "Machine Learning", "estatística", "power bi", "tableau", "data warehouse"
]

class VagasInteligenteFinder:
    def __init__(self, palavras_chave=None):
        self.vagas_encontradas = []
        self.vagas_unicas = {}  # Dicionário para controlar vagas únicas por hash
        self.total_vagas = 0
        self.palavras_chave = palavras_chave or PALAVRAS_CHAVE
        self.setup_selenium()

    def setup_selenium(self):
        """Configura o driver do Selenium"""
        chrome_options = Options()
        chrome_options.add_argument('--headless')
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--disable-dev-shm-usage')
        chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')

        try:
            self.driver = webdriver.Chrome(
                ChromeDriverManager().install(),
                options=chrome_options
            )
            self.driver.set_page_load_timeout(30)
        except Exception as e:
            print(f"Erro ao configurar Selenium: {e}")
            self.driver = None

    def gerar_hash_vaga(self, titulo, empresa, localizacao):
        """Gera um hash único para identificar vagas duplicadas"""
        texto = f"{titulo}_{empresa}_{localizacao}".lower().replace(" ", "")
        return hashlib.md5(texto.encode()).hexdigest()

    def adicionar_vaga_se_unicas(self, vaga):
        """Adiciona uma vaga apenas se for única"""
        vaga_hash = self.gerar_hash_vaga(vaga['titulo'], vaga['empresa'], vaga['localizacao'])

        if vaga_hash not in self.vagas_unicas:
            self.vagas_unicas[vaga_hash] = True
            self.vagas_encontradas.append(vaga)
            self.total_vagas += 1
            return True
        return False

    def buscar_vagas_linkedin(self):
        """Busca vagas no LinkedIn usando scraping"""
        if not self.driver:
            print("Driver do Selenium não disponível para LinkedIn")
            return

        print("Buscando vagas no LinkedIn...")

        try:
            # Buscar por termos mais genéricos primeiro
            termos_busca = ["dados", "data", "analytics", "business intelligence"]

            for termo in termos_busca:
                try:
                    # Construir URL de busca com termos genéricos
                    keywords_encoded = termo.replace(" ", "%20")
                    location_encoded = LOCALIZACAO.replace(" ", "%20").replace(",", "%2C")

                    url = f"https://www.linkedin.com/jobs/search/?keywords={keywords_encoded}&location={location_encoded}&f_TPR=r2592000"

                    self.driver.get(url)
                    time.sleep(5)  # Esperar carregar

                    # Scroll para carregar mais vagas
                    for _ in range(5):
                        self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                        time.sleep(2)

                    # Coletar elementos de vagas
                    vagas = self.driver.find_elements(By.CSS_SELECTOR, '.jobs-search__results-list li')

                    for i, vaga in enumerate(vagas):
                        try:
                            titulo = vaga.find_element(By.CSS_SELECTOR, '.base-search-card__title').text
                            empresa = vaga.find_element(By.CSS_SELECTOR, '.base-search-card__subtitle').text
                            localizacao = vaga.find_element(By.CSS_SELECTOR, '.job-search-card__location').text
                            link = vaga.find_element(By.CSS_SELECTOR, 'a').get_attribute('href')

                            # Verificar se a vaga corresponde às nossas palavras-chave
                            titulo_lower = titulo.lower()
                            corresponde = any(palavra.lower() in titulo_lower for palavra in self.palavras_chave)

                            if corresponde:
                                # A data não é facilmente acessível, usar data atual como aproximação
                                data_publicacao = datetime.now()

                                if data_publicacao >= DATA_INICIO:
                                    nova_vaga = {
                                        'titulo': titulo,
                                        'empresa': empresa,
                                        'localizacao': localizacao,
                                        'data_publicacao': data_publicacao.strftime('%d/%m/%Y'),
                                        'fonte': 'LinkedIn',
                                        'link': link,
                                        'descricao': 'Verifique no site para detalhes completos',
                                        'palavras_chave': ', '.join([p for p in self.palavras_chave if p.lower() in titulo_lower])
                                    }

                                    # Adicionar apenas se for única
                                    self.adicionar_vaga_se_unicas(nova_vaga)
                        except Exception as e:
                            continue

                    print(f"LinkedIn - '{termo}' processado. Total: {self.total_vagas} vagas")
                    time.sleep(3)  # Respeitar o site

                except Exception as e:
                    print(f"Erro ao buscar '{termo}' no LinkedIn: {e}")
                    continue

        except Exception as e:
            print(f"Erro geral ao acessar LinkedIn: {e}")

    def buscar_vagas_indeed(self):
        """Busca vagas no Indeed Brasil"""
        print("Buscando vagas no Indeed Brasil...")

        try:
            base_url = "https://br.indeed.com/jobs"

            # Usar termos genéricos para evitar duplicatas
            termos_busca = ["dados", "analista", "cientista", "business intelligence"]

            for termo in termos_busca:
                try:
                    params = {
                        'q': termo,
                        'l': LOCALIZACAO,
                        'fromage': '7',  # Últimos 7 dias
                        'start': 0
                    }

                    for pagina in range(0, 3):  # 3 páginas por termo
                        params['start'] = pagina * 10
                        headers = {
                            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
                        }

                        response = requests.get(base_url, params=params, headers=headers, timeout=15)

                        if response.status_code != 200:
                            print(f"Erro no Indeed: {response.status_code}")
                            break

                        soup = BeautifulSoup(response.text, 'html.parser')
                        resultados = soup.find_all('div', class_='job_seen_beacon')

                        if not resultados:
                            break

                        for vaga in resultados:
                            try:
                                titulo_elem = vaga.find('h2', class_='jobTitle')
                                titulo = titulo_elem.text.strip() if titulo_elem else 'N/A'

                                # Verificar se a vaga corresponde às nossas palavras-chave
                                titulo_lower = titulo.lower()
                                corresponde = any(palavra.lower() in titulo_lower for palavra in self.palavras_chave)

                                if not corresponde:
                                    continue

                                empresa_elem = vaga.find('span', class_='companyName')
                                empresa = empresa_elem.text.strip() if empresa_elem else 'N/A'

                                local_elem = vaga.find('div', class_='companyLocation')
                                localizacao = local_elem.text.strip() if local_elem else LOCALIZACAO

                                data_elem = vaga.find('span', class_='date')
                                data_texto = data_elem.text.strip() if data_elem else ''

                                # Converter texto de data para objeto datetime
                                data_publicacao = self.parse_data_indeed(data_texto)

                                if data_publicacao >= DATA_INICIO:
                                    link_elem = vaga.find('a', class_='jcs-JobTitle')
                                    link = "https://br.indeed.com" + link_elem['href'] if link_elem else '#'

                                    nova_vaga = {
                                        'titulo': titulo,
                                        'empresa': empresa,
                                        'localizacao': localizacao,
                                        'data_publicacao': data_publicacao.strftime('%d/%m/%Y'),
                                        'fonte': 'Indeed Brasil',
                                        'link': link,
                                        'descricao': 'Verifique no site para detalhes completos',
                                        'palavras_chave': ', '.join([p for p in self.palavras_chave if p.lower() in titulo_lower])
                                    }

                                    # Adicionar apenas se for única
                                    self.adicionar_vaga_se_unicas(nova_vaga)
                            except Exception as e:
                                continue

                        print(f"Indeed - '{termo}' - Página {pagina + 1} processada. Total: {self.total_vagas} vagas")
                        time.sleep(2)  # Respeitar o site

                except Exception as e:
                    print(f"Erro ao buscar '{termo}' no Indeed: {e}")
                    continue

        except Exception as e:
            print(f"Erro geral ao acessar Indeed: {e}")

    def parse_data_indeed(self, data_texto):
        """Converte o texto de data do Indeed para objeto datetime"""
        try:
            data_texto = data_texto.lower()
            hoje = datetime.now()

            if 'hoje' in data_texto or 'just' in data_texto:
                return hoje
            elif 'ontem' in data_texto:
                return hoje - timedelta(days=1)
            elif 'dias' in data_texto:
                num_dias = int(re.search(r'(\d+)', data_texto).group(1))
                return hoje - timedelta(days=num_dias)
            else:
                # Tentar parse de data completa
                try:
                    return datetime.strptime(data_texto, '%d/%m/%Y')
                except:
                    return hoje  # Fallback para data atual
        except:
            return datetime.now()  # Fallback para data atual

    def buscar_vagas_infojobs(self):
        """Busca vagas no InfoJobs"""
        print("Buscando vagas no InfoJobs...")

        try:
            base_url = "https://www.infojobs.com.br/vagas-de-emprego"

            # Usar termos genéricos
            termos_busca = ["dados", "analista", "cientista", "bi"]

            for termo in termos_busca:
                try:
                    params = {
                        'palavraChave': termo,
                        'cidade': 'São Paulo',
                        'pagina': 1
                    }

                    for pagina in range(1, 4):  # 3 páginas por termo
                        params['pagina'] = pagina
                        headers = {
                            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
                        }

                        response = requests.get(base_url, params=params, headers=headers, timeout=15)

                        if response.status_code != 200:
                            print(f"Erro no InfoJobs: {response.status_code}")
                            break

                        soup = BeautifulSoup(response.text, 'html.parser')
                        resultados = soup.select('div.vaga')

                        if not resultados:
                            break

                        for vaga in resultados:
                            try:
                                titulo_elem = vaga.select_one('h2.vagaTitle a')
                                titulo = titulo_elem.text.strip() if titulo_elem else 'N/A'

                                # Verificar se a vaga corresponde às nossas palavras-chave
                                titulo_lower = titulo.lower()
                                corresponde = any(palavra.lower() in titulo_lower for palavra in self.palavras_chave)

                                if not corresponde:
                                    continue

                                empresa_elem = vaga.select_one('span.vaga-company')
                                empresa = empresa_elem.text.strip() if empresa_elem else 'N/A'

                                local_elem = vaga.select_one('span.vaga-location')
                                localizacao = local_elem.text.strip() if local_elem else LOCALIZACAO

                                data_elem = vaga.select_one('span.vaga-date')
                                data_texto = data_elem.text.strip() if data_elem else ''

                                # Converter texto de data para objeto datetime
                                data_publicacao = self.parse_data_infojobs(data_texto)

                                if data_publicacao >= DATA_INICIO:
                                    link = titulo_elem['href'] if titulo_elem else '#'
                                    if not link.startswith('http'):
                                        link = 'https://www.infojobs.com.br' + link

                                    nova_vaga = {
                                        'titulo': titulo,
                                        'empresa': empresa,
                                        'localizacao': localizacao,
                                        'data_publicacao': data_publicacao.strftime('%d/%m/%Y'),
                                        'fonte': 'InfoJobs',
                                        'link': link,
                                        'descricao': 'Verifique no site para detalhes completos',
                                        'palavras_chave': ', '.join([p for p in self.palavras_chave if p.lower() in titulo_lower])
                                    }

                                    # Adicionar apenas se for única
                                    self.adicionar_vaga_se_unicas(nova_vaga)
                            except Exception as e:
                                continue

                        print(f"InfoJobs - '{termo}' - Página {pagina} processada. Total: {self.total_vagas} vagas")
                        time.sleep(2)

                except Exception as e:
                    print(f"Erro ao buscar '{termo}' no InfoJobs: {e}")
                    continue

        except Exception as e:
            print(f"Erro geral ao acessar InfoJobs: {e}")

    def parse_data_infojobs(self, data_texto):
        """Converte o texto de data do InfoJobs para objeto datetime"""
        try:
            # Formato comum: "Publicada em 15/08/2024"
            match = re.search(r'(\d{2}/\d{2}/\d{4})', data_texto)
            if match:
                return datetime.strptime(match.group(1), '%d/%m/%Y')

            # Outros formatos com dias relativos
            data_texto = data_texto.lower()
            hoje = datetime.now()

            if 'hoje' in data_texto:
                return hoje
            elif 'ontem' in data_texto:
                return hoje - timedelta(days=1)
            elif 'dias' in data_texto:
                num_dias = int(re.search(r'(\d+)', data_texto).group(1))
                return hoje - timedelta(days=num_dias)
            else:
                return hoje  # Fallback
        except:
            return datetime.now()  # Fallback

    def buscar_vagas_brasil(self):
        """Busca em sites de empregos brasileiros alternativos"""
        print("Buscando em sites brasileiros alternativos...")

        # Lista de sites para buscar
        sites = [
            {
                'nome': 'Vagas.com',
                'url': "https://www.vagas.com.br/vagas-de-tecnologia-em-sao-paulo",
                'selector': '.vaga',
                'titulo_selector': '.cargo',
                'empresa_selector': '.empr',
                'local_selector': '.vaga-local',
                'data_selector': '.data'
            },
            {
                'nome': 'Catho',
                'url': "https://www.catho.com.br/vagas/tecnologia/em-sao-paulo-sp/",
                'selector': '.job-card',
                'titulo_selector': '.job-card__title',
                'empresa_selector': '.job-card__company',
                'local_selector': '.job-card__location',
                'data_selector': '.job-card__date'
            }
        ]

        for site in sites:
            try:
                headers = {
                    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
                }

                response = requests.get(site['url'], headers=headers, timeout=15)
                if response.status_code != 200:
                    continue

                soup = BeautifulSoup(response.text, 'html.parser')
                vagas = soup.select(site['selector'])

                for vaga in vagas[:20]:  # Limitar a 20 vagas por site
                    try:
                        titulo = vaga.select_one(site['titulo_selector']).text.strip() if vaga.select_one(site['titulo_selector']) else 'N/A'

                        # Verificar se a vaga corresponde às nossas palavras-chave
                        titulo_lower = titulo.lower()
                        corresponde = any(palavra.lower() in titulo_lower for palavra in self.palavras_chave)

                        if not corresponde:
                            continue

                        empresa = vaga.select_one(site['empresa_selector']).text.strip() if vaga.select_one(site['empresa_selector']) else 'N/A'
                        localizacao = vaga.select_one(site['local_selector']).text.strip() if vaga.select_one(site['local_selector']) else LOCALIZACAO

                        # Usar data atual como fallback
                        data_publicacao = datetime.now()

                        nova_vaga = {
                            'titulo': titulo,
                            'empresa': empresa,
                            'localizacao': localizacao,
                            'data_publicacao': data_publicacao.strftime('%d/%m/%Y'),
                            'fonte': site['nome'],
                            'link': site['url'],
                            'descricao': 'Verifique no site para detalhes completos',
                            'palavras_chave': ', '.join([p for p in self.palavras_chave if p.lower() in titulo_lower])
                        }

                        # Adicionar apenas se for única
                        self.adicionar_vaga_se_unicas(nova_vaga)
                    except:
                        continue

                print(f"{site['nome']} processado. Total: {self.total_vagas} vagas")
                time.sleep(2)

            except Exception as e:
                print(f"Erro ao acessar {site['nome']}: {e}")
                continue

    def buscar_todas_vagas(self):
        """Executa todas as buscas"""
        print("Iniciando busca por vagas em São Paulo...")
        print(f"Período: desde {DATA_INICIO.strftime('%d/%m/%Y')} até hoje")
        print(f"Localização: {LOCALIZACAO}")
        print(f"Palavras-chave: {', '.join(self.palavras_chave)}")

        # Executar todas as buscas
        self.buscar_vagas_linkedin()
        self.buscar_vagas_indeed()
        self.buscar_vagas_infojobs()
        self.buscar_vagas_brasil()

        print(f"Busca concluída! Total de {self.total_vagas} vagas únicas encontradas.")

    def exportar_resultados(self):
        """Exporta os resultados para Excel"""
        if not self.vagas_encontradas:
            print("Nenhuma vaga encontrada para exportar")
            return

        df = pd.DataFrame(self.vagas_encontradas)

        # Reordenar colunas
        colunas = ['titulo', 'empresa', 'localizacao', 'data_publicacao', 'fonte', 'palavras_chave', 'descricao', 'link']
        df = df[colunas]

        df.to_excel(OUTPUT_PATH, index=False)
        print(f"Resultados exportados para: {OUTPUT_PATH}")

    def mostrar_resultados(self):
        """Mostra os resultados na tela"""
        if not self.vagas_encontradas:
            print("Nenhuma vaga encontrada")
            return

        print(f"\nVAGAS ENCONTRADAS: {self.total_vagas}")
        print("=" * 200)

        for i, vaga in enumerate(self.vagas_encontradas[:2000], 1):  # Mostrar apenas as 20 primeiras
            print(f"\n{i}. {vaga['titulo']}")
            print(f"   {vaga['empresa']}")
            print(f"   {vaga['localizacao']}")
            print(f"   {vaga['data_publicacao']}")
            print(f"   {vaga['fonte']}")
            print(f"   {vaga['palavras_chave']}")
            print(f"   {vaga['link']}")
            print("-" * 100)

        if self.total_vagas > 20:
            print(f"\n... e mais {self.total_vagas - 2000} vagas (ver arquivo Excel para lista completa)")

# Executar a busca
if __name__ == "__main__":
    finder = VagasInteligenteFinder(palavras_chave=PALAVRAS_CHAVE)
    finder.buscar_todas_vagas()
    finder.mostrar_resultados()
    finder.exportar_resultados()

    # Fechar o driver do Selenium se existir
    if hasattr(finder, 'driver') and finder.driver:
        finder.driver.quit()

Configurando ambiente...
Ambiente configurado!
Importando bibliotecas...
Erro ao configurar Selenium: WebDriver.__init__() got multiple values for argument 'options'
Iniciando busca por vagas em São Paulo...
Período: desde 01/08/2024 até hoje
Localização: São Paulo, SP
Palavras-chave: analista de dados, data analyst, cientista de dados, data scientist, sql, python, business intelligence, preditivo, predictive, modelagem, etl, Manipulação de Dados, Excel, Big Data, Machine Learning, estatística, power bi, tableau, data warehouse
Driver do Selenium não disponível para LinkedIn
Buscando vagas no Indeed Brasil...
Erro no Indeed: 403
Erro no Indeed: 403
Erro no Indeed: 403
Erro no Indeed: 403
Buscando vagas no InfoJobs...
Buscando em sites brasileiros alternativos...
Vagas.com processado. Total: 0 vagas
Busca concluída! Total de 0 vagas únicas encontradas.
Nenhuma vaga encontrada
Nenhuma vaga encontrada para exportar


In [ ]:
# -*- coding: utf-8 -*-
"""
Sistema de Busca de Vagas com APIs Oficiais
Autor: Assistente AI
Data: 2025
"""

print("🔧 Configurando ambiente...")

# Instalar bibliotecas necessárias
!pip install requests pandas datetime python-dotenv --quiet

print("Ambiente configurado!")
print("Importando bibliotecas...")

import requests
import pandas as pd
from datetime import datetime, timedelta
import time
import os
from dotenv import load_dotenv
import json

# Carregar variáveis de ambiente
load_dotenv()

# Configurações
OUTPUT_PATH = "/content/sample_data/vagas_apis_oficiais.xlsx"
DATA_INICIO = datetime(2024, 8, 1)  # 01/08/2024
LOCALIZACAO = "São Paulo, SP"

# Palavras-chave prioritárias para área de dados
PALAVRAS_CHAVE = [
    "analista de dados", "data analyst", "cientista de dados", "data scientist",
    "sql", "python", "business intelligence", "preditivo", "predictive",
    "modelagem", "etl", "Manipulação de Dados", "Excel", "Big Data",
    "Machine Learning", "estatística", "power bi", "tableau", "data warehouse"
]

class APIVagasFinder:
    def __init__(self, palavras_chave=None):
        self.vagas_encontradas = []
        self.total_vagas = 0
        self.palavras_chave = palavras_chave or PALAVRAS_CHAVE

        # Configurar credenciais de APIs
        self.infojobs_client_id = os.getenv('INFOJOBS_CLIENT_ID', 'seu_client_id_aqui')
        self.infojobs_client_secret = os.getenv('INFOJOBS_CLIENT_SECRET', 'seu_client_secret_aqui')
        self.serpapi_key = os.getenv('SERPAPI_KEY', 'sua_chave_serpapi_aqui')

    def buscar_vagas_infojobs_api(self):
        """Busca vagas usando a API oficial do InfoJobs :cite[1]:cite[5]:cite[8]"""
        print("Buscando vagas via API do InfoJobs...")

        try:
            # Autenticação na API do InfoJobs
            auth_url = "https://www.infojobs.com.br/oauth/authorize"
            token_url = "https://www.infojobs.com.br/oauth/token"

            # Obter token de acesso (simplificado - na prática precisa do fluxo OAuth completo)
            # Para uso real, implemente o fluxo OAuth2 conforme documentação
            auth_params = {
                'grant_type': 'client_credentials',
                'client_id': self.infojobs_client_id,
                'client_secret': self.infojobs_client_secret,
                'scope': 'api_candidatura'
            }

            # Para demonstração, vamos usar o endpoint de busca pública
            for palavra_chave in self.palavras_chave[:5]:  # Limitar a 5 palavras-chave
                try:
                    url = "https://api.infojobs.net/api/7/offer"
                    params = {
                        'q': palavra_chave,
                        'city': 'São Paulo',
                        'sortRelevance': 'true',
                        'maxResults': 50
                    }

                    headers = {
                        'Authorization': f'Basic {self.infojobs_client_id}:{self.infojobs_client_secret}',
                        'Content-Type': 'application/json',
                        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
                    }

                    response = requests.get(url, params=params, headers=headers, timeout=15)

                    if response.status_code == 200:
                        dados = response.json()

                        if 'items' in dados:
                            for vaga in dados['items']:
                                try:
                                    # Converter data para objeto datetime
                                    data_texto = vaga.get('creationDate', '')
                                    data_publicacao = datetime.strptime(data_texto, '%Y-%m-%dT%H:%M:%S.%fZ') if data_texto else datetime.now()

                                    if data_publicacao >= DATA_INICIO:
                                        self.vagas_encontradas.append({
                                            'titulo': vaga.get('title', 'N/A'),
                                            'empresa': vaga.get('author', {}).get('name', 'N/A'),
                                            'localizacao': vaga.get('city', LOCALIZACAO),
                                            'data_publicacao': data_publicacao.strftime('%d/%m/%Y'),
                                            'fonte': 'InfoJobs API',
                                            'link': vaga.get('link', '#'),
                                            'descricao': vaga.get('description', '')[:200] + '...' if vaga.get('description') else 'N/A',
                                            'palavra_chave': palavra_chave
                                        })
                                        self.total_vagas += 1
                                except Exception as e:
                                    continue

                            print(f"InfoJobs API - '{palavra_chave}' processada. Total: {self.total_vagas} vagas")
                        else:
                            print(f"Nenhum resultado encontrado no InfoJobs API para '{palavra_chave}'")

                    time.sleep(1)  # Respeitar a API

                except Exception as e:
                    print(f"Erro ao buscar '{palavra_chave}' no InfoJobs API: {e}")
                    continue

        except Exception as e:
            print(f"Erro geral ao acessar InfoJobs API: {e}")

    def buscar_vagas_google_jobs_api(self):
        """Busca vagas usando Google Jobs API através do SerpApi"""
        print("Buscando vagas via Google Jobs API...")

        try:
            if self.serpapi_key == 'sua_chave_serpapi_aqui':
                print("Chave SerpAPI não configurada. Pulando Google Jobs API.")
                return

            # Para cada palavra-chave, fazer uma busca
            for palavra_chave in self.palavras_chave[:3]:  # Limitar a 3 palavras-chave
                try:
                    params = {
                        "engine": "google_jobs",
                        "q": f"{palavra_chave} {LOCALIZACAO}",
                        "hl": "pt",
                        "gl": "br",
                        "api_key": self.serpapi_key
                    }

                    search_url = "https://serpapi.com/search"
                    response = requests.get(search_url, params=params, timeout=15)

                    if response.status_code == 200:
                        results = response.json()

                        if "jobs_results" in results:
                            for vaga in results["jobs_results"]:
                                try:
                                    # Converter data para objeto datetime
                                    data_texto = vaga.get("detected_extensions", {}).get("posted_at", "")
                                    data_publicacao = self.parse_data_google_jobs(data_texto)

                                    if data_publicacao >= DATA_INICIO:
                                        self.vagas_encontradas.append({
                                            'titulo': vaga.get('title', 'N/A'),
                                            'empresa': vaga.get('company_name', 'N/A'),
                                            'localizacao': vaga.get('location', LOCALIZACAO),
                                            'data_publicacao': data_publicacao.strftime('%d/%m/%Y'),
                                            'fonte': 'Google Jobs API',
                                            'link': vaga.get('related_links', [{}])[0].get('link', '#') if vaga.get('related_links') else '#',
                                            'descricao': vaga.get('description', '')[:200] + '...' if vaga.get('description') else 'N/A',
                                            'palavra_chave': palavra_chave
                                        })
                                        self.total_vagas += 1
                                except Exception as e:
                                    continue

                            print(f"Google Jobs API - '{palavra_chave}' processada. Total: {self.total_vagas} vagas")
                        else:
                            print(f"Nenhum resultado encontrado no Google Jobs API para '{palavra_chave}'")

                    time.sleep(2)  # Respeitar a API

                except Exception as e:
                    print(f"Erro ao buscar '{palavra_chave}' no Google Jobs API: {e}")
                    continue

        except Exception as e:
            print(f"Erro geral ao acessar Google Jobs API: {e}")

    def parse_data_google_jobs(self, data_texto):
        """Converte o texto de data do Google Jobs para objeto datetime"""
        try:
            data_texto = data_texto.lower()
            hoje = datetime.now()

            if 'hour' in data_texto or 'hora' in data_texto:
                return hoje
            elif 'day' in data_texto or 'dia' in data_texto:
                num_dias = int(re.search(r'(\d+)', data_texto).group(1))
                return hoje - timedelta(days=num_dias)
            elif 'week' in data_texto or 'semana' in data_texto:
                num_semanas = int(re.search(r'(\d+)', data_texto).group(1))
                return hoje - timedelta(weeks=num_semanas)
            else:
                return hoje  # Fallback para data atual
        except:
            return datetime.now()  # Fallback para data atual

    def buscar_vagas_adzuna_api(self):
        """Busca vagas usando a API do Adzuna"""
        print("Buscando vagas via Adzuna API...")

        try:
            # Configuração da API Adzuna (requer registro)
            app_id = os.getenv('ADZUNA_APP_ID', 'seu_app_id_aqui')
            app_key = os.getenv('ADZUNA_APP_KEY', 'seu_app_key_aqui')
            pais = "br"  # Brasil

            if app_id == 'seu_app_id_aqui' or app_key == 'seu_app_key_aqui':
                print("Credenciais Adzuna não configuradas. Pulando Adzuna API.")
                return

            for palavra_chave in self.palavras_chave[:3]:  # Limitar a 3 palavras-chave
                try:
                    url = f"https://api.adzuna.com/v1/api/jobs/{pais}/search/1"
                    params = {
                        'app_id': app_id,
                        'app_key': app_key,
                        'results_per_page': 20,
                        'what': palavra_chave,
                        'where': 'São Paulo',
                        'content-type': 'application/json'
                    }

                    response = requests.get(url, params=params, timeout=15)

                    if response.status_code == 200:
                        dados = response.json()

                        if 'results' in dados:
                            for vaga in dados['results']:
                                try:
                                    # Converter data para objeto datetime
                                    data_texto = vaga.get('created', '')
                                    data_publicacao = datetime.strptime(data_texto, '%Y-%m-%dT%H:%M:%SZ') if data_texto else datetime.now()

                                    if data_publicacao >= DATA_INICIO:
                                        self.vagas_encontradas.append({
                                            'titulo': vaga.get('title', 'N/A'),
                                            'empresa': vaga.get('company', {}).get('display_name', 'N/A'),
                                            'localizacao': vaga.get('location', {}).get('display_name', LOCALIZACAO),
                                            'data_publicacao': data_publicacao.strftime('%d/%m/%Y'),
                                            'fonte': 'Adzuna API',
                                            'link': vaga.get('redirect_url', '#'),
                                            'descricao': vaga.get('description', '')[:200] + '...' if vaga.get('description') else 'N/A',
                                            'palavra_chave': palavra_chave
                                        })
                                        self.total_vagas += 1
                                except Exception as e:
                                    continue

                            print(f"Adzuna API - '{palavra_chave}' processada. Total: {self.total_vagas} vagas")
                        else:
                            print(f"Nenhum resultado encontrado no Adzuna API para '{palavra_chave}'")

                    time.sleep(1)  # Respeitar a API

                except Exception as e:
                    print(f"Erro ao buscar '{palavra_chave}' no Adzuna API: {e}")
                    continue

        except Exception as e:
            print(f"Erro geral ao acessar Adzuna API: {e}")

    def buscar_vagas_github_jobs_api(self):
        """Busca vagas usando a API do GitHub Jobs"""
        print("Buscando vagas via GitHub Jobs API...")

        try:
            for palavra_chave in self.palavras_chave[:3]:  # Limitar a 3 palavras-chave
                try:
                    url = "https://jobs.github.com/positions.json"
                    params = {
                        'description': palavra_chave,
                        'location': 'São Paulo'
                    }

                    response = requests.get(url, params=params, timeout=15)

                    if response.status_code == 200:
                        vagas = response.json()

                        for vaga in vagas:
                            try:
                                # Converter data para objeto datetime
                                data_texto = vaga.get('created_at', '')
                                data_publicacao = datetime.strptime(data_texto, '%a %b %d %H:%M:%S UTC %Y') if data_texto else datetime.now()

                                if data_publicacao >= DATA_INICIO:
                                    self.vagas_encontradas.append({
                                        'titulo': vaga.get('title', 'N/A'),
                                        'empresa': vaga.get('company', 'N/A'),
                                        'localizacao': vaga.get('location', LOCALIZACAO),
                                        'data_publicacao': data_publicacao.strftime('%d/%m/%Y'),
                                        'fonte': 'GitHub Jobs API',
                                        'link': vaga.get('url', '#'),
                                        'descricao': vaga.get('description', '')[:200] + '...' if vaga.get('description') else 'N/A',
                                        'palavra_chave': palavra_chave
                                    })
                                    self.total_vagas += 1
                            except Exception as e:
                                continue

                        print(f"GitHub Jobs API - '{palavra_chave}' processada. Total: {self.total_vagas} vagas")
                    else:
                        print(f"Nenhum resultado encontrado no GitHub Jobs API para '{palavra_chave}'")

                    time.sleep(1)  # Respeitar a API

                except Exception as e:
                    print(f"Erro ao buscar '{palavra_chave}' no GitHub Jobs API: {e}")
                    continue

        except Exception as e:
            print(f"Erro geral ao acessar GitHub Jobs API: {e}")

    def remover_duplicatas(self):
        """Remove vagas duplicadas com base no título e empresa"""
        if not self.vagas_encontradas:
            return

        # Usar pandas para remover duplicatas
        df = pd.DataFrame(self.vagas_encontradas)
        df = df.drop_duplicates(subset=['titulo', 'empresa'], keep='first')
        self.vagas_encontradas = df.to_dict('records')
        self.total_vagas = len(self.vagas_encontradas)

    def buscar_todas_vagas(self):
        """Executa todas as buscas"""
        print("Iniciando busca por vagas em São Paulo usando APIs...")
        print(f"Período: desde {DATA_INICIO.strftime('%d/%m/%Y')} até hoje")
        print(f"Localização: {LOCALIZACAO}")
        print(f"Palavras-chave: {', '.join(self.palavras_chave)}")

        # Executar todas as buscas por APIs
        self.buscar_vagas_infojobs_api()
        self.buscar_vagas_google_jobs_api()
        self.buscar_vagas_adzuna_api()
        self.buscar_vagas_github_jobs_api()

        # Remover duplicatas
        self.remover_duplicatas()

        print(f"Busca concluída! Total de {self.total_vagas} vagas únicas encontradas.")

    def exportar_resultados(self):
        """Exporta os resultados para Excel"""
        if not self.vagas_encontradas:
            print("Nenhuma vaga encontrada para exportar")
            return

        df = pd.DataFrame(self.vagas_encontradas)

        # Reordenar colunas
        colunas = ['titulo', 'empresa', 'localizacao', 'data_publicacao', 'fonte', 'palavra_chave', 'descricao', 'link']
        df = df[colunas]

        df.to_excel(OUTPUT_PATH, index=False)
        print(f"Resultados exportados para: {OUTPUT_PATH}")

    def mostrar_resultados(self):
        """Mostra os resultados na tela"""
        if not self.vagas_encontradas:
            print("Nenhuma vaga encontrada")
            return

        print(f"\nVAGAS ENCONTRADAS: {self.total_vagas}")
        print("=" * 100)

        for i, vaga in enumerate(self.vagas_encontradas[:15], 1):  # Mostrar apenas as 15 primeiras
            print(f"\n{i}. {vaga['titulo']}")
            print(f"   {vaga['empresa']}")
            print(f"   {vaga['localizacao']}")
            print(f"   {vaga['data_publicacao']}")
            print(f"   {vaga['fonte']}")
            print(f"   {vaga['palavra_chave']}")
            print(f"   {vaga['descricao']}")
            print(f"   {vaga['link']}")
            print("-" * 100)

        if self.total_vagas > 15:
            print(f"\n... e mais {self.total_vagas - 15} vagas (ver arquivo Excel para lista completa)")

# Executar a busca
if __name__ == "__main__":
    finder = APIVagasFinder(palavras_chave=PALAVRAS_CHAVE)
    finder.buscar_todas_vagas()
    finder.mostrar_resultados()
    finder.exportar_resultados()

🔧 Configurando ambiente...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.4/264.4 kB 9.3 MB/s eta 0:00:00
Ambiente configurado!
Importando bibliotecas...
Iniciando busca por vagas em São Paulo usando APIs...
Período: desde 01/08/2024 até hoje
Localização: São Paulo, SP
Palavras-chave: analista de dados, data analyst, cientista de dados, data scientist, sql, python, business intelligence, preditivo, predictive, modelagem, etl, Manipulação de Dados, Excel, Big Data, Machine Learning, estatística, power bi, tableau, data warehouse
Buscando vagas via API do InfoJobs...
Buscando vagas via Google Jobs API...
 Chave SerpAPI não configurada. Pulando Google Jobs API.
Buscando vagas via Adzuna API...
 Credenciais Adzuna não configuradas. Pulando Adzuna API.
Buscando vagas via GitHub Jobs API...
Erro ao buscar 'analista de dados' no GitHub 